In [37]:
# Import the libraries
from pymongo import MongoClient
import requests
from dotenv import load_dotenv
import os
import json
import folium
from folium.vector_layers import Circle

In [38]:
# Access to the database in MongoDB
client=MongoClient()
db=client.get_database("companies")

In [39]:
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [40]:
load_dotenv()

True

In [41]:
# Get the tokens that were saved in the .env
gp_token=os.getenv("GOOGLE_P_TOKEN")
fq_client_id=os.getenv("FQ_CLIENT_ID")
fq_client_secret=os.getenv("FQ_CLIENT_SECRET")

# First screening with MongoDB operator

- I used $near operator to select the companies that are in Silicon Valley (California), because the big tech companies are stablished there.
- I also searched for companies that were near the airport, I looked for companies near the San Francisco airport but as there was only one, I then looked for companies near the other big airport in that area which is the San José airport.

In [70]:
# Create a new database named companies_unwinded, so that I do not 
# have several offices for the same company

res = db.companies.aggregate([
                      {"$unwind":"$offices" },
                      {"$match":{"offices.latitude":{"$ne":None}}},
                      {"$match":{"offices.longitude":{"$ne":None}}},
                      {"$project":{"_id":0}}])

db.companies_unwinded.insert_many(res)

offices = db.companies_unwinded

res = offices.find({},{"offices":1})


for c in res:
    filt = {"_id":c["_id"]}
    coord = {
              "type": "Point",
              "coordinates": [c["offices"]["longitude"], c["offices"]["latitude"]]
            }
    update = {"$set":{"coord":coord}}
    offices.update_one(filt,update)


In [20]:
type(res)

pymongo.cursor.Cursor

In [21]:
# Look for the companies that are near San Francisco airport

point = {"type":"Point",
         "coordinates":[-122.390091, 37.615404]}

query ={
            "coord":{
                "$near":{
                    "$geometry":point,
                    "$maxDistance":2500,
                    #"$minDistance":
                }
            }
        }

res = db.companies_unwinded.find(query,{"offices":1})
list(res)

# There are only two so I look for the companies near San José airport

[{'_id': ObjectId('5fafa246e21df4eac0a71b53'),
  'offices': {'description': 'Corporate Headquarters',
   'address1': '1745 Adrian Rd. Suite 1',
   'address2': '',
   'zip_code': '94010',
   'city': 'Burlingame',
   'state_code': 'CA',
   'country_code': 'USA',
   'latitude': 37.598478,
   'longitude': -122.376572}},
 {'_id': ObjectId('5fb012d0f93b0b3e63f748a2'),
  'offices': {'description': 'Corporate Headquarters',
   'address1': '1745 Adrian Rd. Suite 1',
   'address2': '',
   'zip_code': '94010',
   'city': 'Burlingame',
   'state_code': 'CA',
   'country_code': 'USA',
   'latitude': 37.598478,
   'longitude': -122.376572}}]

In [104]:
# Look for the companies near San José airport


point = {"type":"Point",
         "coordinates":[-121.928927, 37.363803]}

query ={
            "coord":{
                "$near":{
                    "$geometry":point,
                    "$maxDistance":1000,
                    #"$minDistance":
                }
            }
        }


res1 = db.companies_unwinded.find(query, {"offices":1})

# There are many companies that are at 1km or less from this airport

In [102]:
len(list(res1))

21

In [107]:
# lat and long first company
lat=[]
lng=[]
for x in range(0,21):
    latitude=res1[x]["offices"]["latitude"]
    longitude=res1[x]["offices"]["longitude"]
    lat.append(latitude)
    lng.append(longitude)  
print(set(lat))
print(set(lng))

{37.3709977, 37.371088, 37.3706126, 37.37027, 37.3721771}
{-121.925355, -121.9250398, -121.9219538, -121.9235529, -121.924053}


I realized that there are some companies that are duplicated, so the total number of companies is reduced to 5.

# First company analysis

In [47]:
def api_function(ll, query, name):
    url = 'https://api.foursquare.com/v2/venues/explore'

    params = dict(
    client_id=fq_client_id,
    client_secret=fq_client_secret,
    v='20201015',
    ll=ll,
    query=query,
    limit=15
    )
    resp = requests.get(url=url, params=params)
    name = json.loads(resp.text)
    return name

In [44]:
# I selected the first company on the list above which is located in Sunnyvale, an area that is near to Google and Facebook offices
# I make a query to the Foursquare API to look for tech startups near this company.
tech1=api_function('37.3709977,-121.9235529', 'Tech Startup', 'tech_startups_sun1')
len(list(tech1))

2

In [91]:
def dist(lista):
    distances=[]
    for x in range(0,14):
        result=lista["response"]["groups"][0]["items"][x]["venue"]["location"]["distance"]
        distances.append(result)
    return distances

dist(tech1)

[544, 171, 170, 229, 256, 276, 367, 368, 434, 446, 485, 596, 181, 608]

In [92]:
def lat(lista):
    latitude=[]
    for x in range(0,14):
        result=lista["response"]["groups"][0]["items"][x]["venue"]["location"]["lat"]
        latitude.append(result)
    return latitude
lat(tech1)

[37.375874679466456,
 37.370972648019674,
 37.36947,
 37.36957044583222,
 37.3695173976203,
 37.369302000000005,
 37.37009996545508,
 37.371297,
 37.369042,
 37.36805224766053,
 37.36753085454393,
 37.368907,
 37.37114012,
 37.36766592556348]

In [93]:
def lng(lista):
    longitude=[]
    for x in range(0,14):
        result=lista["response"]["groups"][0]["items"][x]["venue"]["location"]["lng"]
        longitude.append(result)
    return longitude

lng(tech1)

[-121.92306376202592,
 -121.92549178587224,
 -121.92351,
 -121.921677387159,
 -121.92133577652343,
 -121.92127099999999,
 -121.91955220035622,
 -121.9194084405899,
 -121.919304,
 -121.92011894731519,
 -121.92021896118176,
 -121.917341,
 -121.92559373,
 -121.9180963651028]

In [94]:
def name(lista):
    names=[]
    for x in range(0,14):
        result=lista["response"]["groups"][0]["items"][x]["venue"]["name"]
        names.append(result)
    return names

name(tech1)

['PayPal',
 'SourceN HQ',
 'FixStream',
 'River Meadow Software',
 'Robin Systems',
 'PlayPhone, Inc.',
 'skytree',
 'Ocarina Networks',
 'SVACPA',
 'Gaia Online',
 'Ensighten',
 'GEO Seminconductor',
 'DataTech Labs Data Recovery',
 'Nutanix HQ']

In [55]:
# I search for the nearest tech startups to the offices of the 
# company I selected
dist_tech=tech1["response"]["groups"][0]["items"][0]["venue"]["location"]["distance"]
lat_tech=tech1["response"]["groups"][0]["items"][0]["venue"]["location"]["lat"]
lng_tech=tech1["response"]["groups"][0]["items"][0]["venue"]["location"]["lng"]
name_tech=tech1["response"]["groups"][0]["items"][0]["venue"]["name"]

print(f"company name: {name_tech}, company coordinates: {lat_tech},{lng_tech}, distance to the offices: {dist_tech}")

company name: PayPal, company coordinates: 37.375874679466456,-121.92306376202592, distance to the offices: 544


The nearest company is PayPal which is at 544m from the offices.

In [61]:
# I make a query to the Foursquare API to look for design studios near this company.

design1=api_function('37.3709977,-121.9235529', 'Design Studio', 'design_studio_sun1')


In [173]:
print(dist(design1))
print("----------------------------------------------")
print(lat(design1))
print("----------------------------------------------")
print(lng(design1))
print("----------------------------------------------")
print(name(design1))

[298, 785, 582, 1274, 1291, 1345, 1351, 1359, 1465, 1490, 1587, 1614, 1765, 1401]
----------------------------------------------
[37.37334604119023, 37.3672899, 37.3691219, 37.37102127075195, 37.378426714690484, 37.3707160949707, 37.374528, 37.373359, 37.374013, 37.379857, 37.373688, 37.373843, 37.383968353271484, 37.3657117]
----------------------------------------------
[-121.92193508148193, -121.916, -121.917411, -121.90914154052734, -121.91234536963067, -121.9083480834961, -121.908934, -121.90847, -121.907427, -121.910925, -121.905933, -121.905661, -121.91206359863281, -121.9091752]
----------------------------------------------
['Kiskon Technologies', 'Watry Design', 'Konica Minolta Business Solutions', 'Screened Magic', 'Home Expo &  Design', 'Woodenbridge', 'Summit Cabinet', 'The Ceramic Corporation', 'Atlas Glass', 'Bay Stone', 'Granite & Stone Expo', 'Zeus Tile', 'Emser Tile&Natural Stone', 'Signarama San Jose']


In [59]:
dist_design=design1["response"]["groups"][0]["items"][0]["venue"]["location"]["distance"]
lat_design=design1["response"]["groups"][0]["items"][0]["venue"]["location"]["lat"]
lng_design=design1["response"]["groups"][0]["items"][0]["venue"]["location"]["lng"]
name_design=design1["response"]["groups"][0]["items"][0]["venue"]["name"]

print(f"company name: {name_design}, company coordinates: {lat_design},{lng_design}, distance to the offices: {dist_design}")

dist_design=design1["response"]["groups"][0]["items"][2]["venue"]["location"]["distance"]
lat_design=design1["response"]["groups"][0]["items"][2]["venue"]["location"]["lat"]
lng_design=design1["response"]["groups"][0]["items"][2]["venue"]["location"]["lng"]
name_design=design1["response"]["groups"][0]["items"][2]["venue"]["name"]

print(f"company name: {name_design}, company coordinates: {lat_design},{lng_design}, distance to the offices: {dist_design}")

company name: Kiskon Technologies, company coordinates: 37.37334604119023,-121.92193508148193, distance to the offices: 298
company name: Konica Minolta Business Solutions, company coordinates: 37.3691219,-121.917411, distance to the offices: 582


The nearest studio is at 298m and there is another one at 582m from the office.

In [62]:
# I make a query to the Foursquare API to look for schools near this company.

sch1=api_function('37.3709977,-121.9235529', 'Elementary School', 'elementary_sch_sun1')


In [182]:
print(dist(sch1))
print("----------------------------------------------")
print(lat(sch1))
print("----------------------------------------------")
print(lng(sch1))
print("----------------------------------------------")
print(name(sch1))

[1280, 2768, 2901, 3224, 3362, 3460, 3481, 3140, 3658, 3856, 3967, 3675, 4504, 4514]
----------------------------------------------
[37.36330687882652, 37.388926007147695, 37.38955552112065, 37.39309384415278, 37.358551025390625, 37.343393216725616, 37.35738251403415, 37.347336, 37.348379, 37.350991, 37.38721347682061, 37.398261959325076, 37.40608596801758, 37.333647875243265]
----------------------------------------------
[-121.9127954630632, -121.90186142921448, -121.94658055857437, -121.94713050142707, -121.95817565917969, -121.94153776982016, -121.9589768845584, -121.942885, -121.893555, -121.88796799999999, -121.88360782691444, -121.94699718220039, -121.898193359375, -121.94342527769899]
----------------------------------------------
['Bachrodt Elementary School', 'Orchard Elementary School', 'Montague Elementary School', 'Don Callejon School', 'Jano Hindi Bee', 'Washington Open Elementary', 'Scott Lane Elementary School', "St Clare's Catholic School", 'Suzume No Gakko', 'Grant El

In [57]:
dist_school=sch1["response"]["groups"][0]["items"][0]["venue"]["location"]["distance"]
lat_school=sch1["response"]["groups"][0]["items"][0]["venue"]["location"]["lat"]
lng_school=sch1["response"]["groups"][0]["items"][0]["venue"]["location"]["lng"]
name_school=sch1["response"]["groups"][0]["items"][0]["venue"]["name"]

print(f"company name: {name_school}, company coordinates: {lat_school},{lng_school}, distance to the offices: {dist_school}")

company name: Bachrodt Elementary School, company coordinates: 37.36330687882652,-121.9127954630632, distance to the offices: 1280


The nearest school is at 1.2km

In [63]:
# I make a query to the Foursquare API to look for nighlife spots near this company.

night1=api_function('37.3709977,-121.9235529', 'Nightlife Spot', 'nightlife_spot_sun1')


In [179]:
print(dist(night1))
print("----------------------------------------------")
print(lat(night1))
print("----------------------------------------------")
print(lng(night1))
print("----------------------------------------------")
print(name(night1))

[298, 785, 582, 1274, 1291, 1345, 1351, 1359, 1465, 1490, 1587, 1614, 1765, 1401]
----------------------------------------------
[37.37334604119023, 37.3672899, 37.3691219, 37.37102127075195, 37.378426714690484, 37.3707160949707, 37.374528, 37.373359, 37.374013, 37.379857, 37.373688, 37.373843, 37.383968353271484, 37.3657117]
----------------------------------------------
[-121.92193508148193, -121.916, -121.917411, -121.90914154052734, -121.91234536963067, -121.9083480834961, -121.908934, -121.90847, -121.907427, -121.910925, -121.905933, -121.905661, -121.91206359863281, -121.9091752]
----------------------------------------------
['Kiskon Technologies', 'Watry Design', 'Konica Minolta Business Solutions', 'Screened Magic', 'Home Expo &  Design', 'Woodenbridge', 'Summit Cabinet', 'The Ceramic Corporation', 'Atlas Glass', 'Bay Stone', 'Granite & Stone Expo', 'Zeus Tile', 'Emser Tile&Natural Stone', 'Signarama San Jose']


In [69]:
dist_party=night1["response"]["groups"][0]["items"][5]["venue"]["location"]["distance"]
lat_party=night1["response"]["groups"][0]["items"][5]["venue"]["location"]["lat"]
lng_party=night1["response"]["groups"][0]["items"][5]["venue"]["location"]["lng"]
name_party=night1["response"]["groups"][0]["items"][5]["venue"]["name"]

print(f"company name: {name_party}, company coordinates: {lat_party},{lng_party}, distance to the offices: {dist_party}")

dist_party=night1["response"]["groups"][0]["items"][9]["venue"]["location"]["distance"]
lat_party=night1["response"]["groups"][0]["items"][9]["venue"]["location"]["lat"]
lng_party=night1["response"]["groups"][0]["items"][9]["venue"]["location"]["lng"]
name_party=night1["response"]["groups"][0]["items"][9]["venue"]["name"]

print(f"company name: {name_party}, company coordinates: {lat_party},{lng_party}, distance to the offices: {dist_party}")


company name: Doubletree Pool, company coordinates: 37.37152348661807,-121.92232810486176, distance to the offices: 123
company name: 2050 Lobby Lounge & Bar, company coordinates: 37.371769553739405,-121.92226542378253, distance to the offices: 142


There are two nighlife sports near, one at 123m and other at 142m

In [ ]:
# I make a query to the Foursquare API to look for vegetarian restaurants near this company.

veg1=api_function('37.3709977,-121.9235529', 'Vegan Restaurant', 'veg_rest_sun1')


In [183]:
print(dist(veg1))
print("----------------------------------------------")
print(lat(veg1))
print("----------------------------------------------")
print(lng(veg1))
print("----------------------------------------------")
print(name(veg1))

[3051, 3394, 5320, 2120, 2619, 2436, 3399, 3762, 3955, 4061, 4145, 4858, 5206, 5589]
----------------------------------------------
[37.34460383642609, 37.343718446032305, 37.32614245482791, 37.385706, 37.35659, 37.3607916, 37.34049606323242, 37.33930772, 37.338702000000005, 37.404856, 37.396342, 37.41086084497713, 37.35198682571322, 37.42087245286653]
----------------------------------------------
[-121.93286117841694, -121.9406861466985, -121.94432964084655, -121.908328, -121.94696, -121.8991962, -121.92169952392578, -121.93835143, -121.942205, -121.906444, -121.88920700000001, -121.94593038741806, -121.97731920128297, -121.91625220186718]
----------------------------------------------
["Ike's Sandwiches", "deedee's", 'Loving Hut', 'Vegefarm', 'The Happy Hooligans', "Brother Baby's BBQ", 'Queen Of Sheba', 'Raw Vittles', 'Clean Eating with Katie', 'Tasty Vegetarian Cuisine', 'SJ Vegetarian', "Sri's Kitchen", 'Peacock Indian Cuisine', 'Loving Hut']


In [74]:
dist_veg=veg1["response"]["groups"][0]["items"][3]["venue"]["location"]["distance"]
lat_veg=veg1["response"]["groups"][0]["items"][3]["venue"]["location"]["lat"]
lng_veg=veg1["response"]["groups"][0]["items"][3]["venue"]["location"]["lng"]
name_veg=veg1["response"]["groups"][0]["items"][3]["venue"]["name"]

print(f"company name: {name_veg}, company coordinates: {lat_veg},{lng_veg}, distance to the offices: {dist_veg}")


company name: Brother Baby's BBQ, company coordinates: 37.3607916,-121.8991962, distance to the offices: 2436


The nearest restaurant is at 2.4km from the office.

In [ ]:
# I make a query to the Foursquare API to look for basketball courts near this company.

basket1=api_function('37.3709977,-121.9235529', 'Basketball Court', 'basketball_court_sun1')


In [186]:
print(dist(basket1))
print("----------------------------------------------")
print(lat(basket1))
print("----------------------------------------------")
print(lng(basket1))
print("----------------------------------------------")
print(name(basket1))

[3710, 668, 1036, 1071, 1119, 2757, 3022, 3556, 3970, 4091, 4679, 4265, 4483, 4794]
----------------------------------------------
[37.33779578622328, 37.37696075439453, 37.36441421508789, 37.37117385864258, 37.364402, 37.3881680172003, 37.377344606195855, 37.39918075740896, 37.37372, 37.3753662109375, 37.40387815016547, 37.33270914406301, 37.330723171109355, 37.3980712890625]
----------------------------------------------
[-121.91989631992244, -121.92266082763672, -121.91527557373047, -121.91143798828125, -121.913997, -121.94602505794903, -121.9567734963286, -121.94248645305073, -121.968306, -121.96947479248047, -121.9565124356151, -121.9253028797591, -121.92286373936807, -121.88139343261719]
----------------------------------------------
['Central YMCA', 'eBay Inc. Basketball Court', 'Brocade Courts', 'Basketball Gym', 'Skyport!', 'Montague Park', '3 Ring', 'mansion grove basketball court', "Rucker's Basketball Court", 'Podium Golden State Warriors Basketball Court', 'Lick Mill Park'

In [86]:
dist_basket=basket1["response"]["groups"][0]["items"][1]["venue"]["location"]["distance"]
lat_basket=basket1["response"]["groups"][0]["items"][1]["venue"]["location"]["lat"]
lng_basket=basket1["response"]["groups"][0]["items"][1]["venue"]["location"]["lng"]
name_basket=basket1["response"]["groups"][0]["items"][1]["venue"]["name"]

print(f"company name: {name_basket}, company coordinates: {lat_basket},{lng_basket}, distance to the offices: {dist_basket}")

company name: eBay Inc. Basketball Court, company coordinates: 37.37696075439453,-121.92266082763672, distance to the offices: 668


The nearest one is at 668m

In [ ]:
# I make a query to the Foursquare API to look for Starbucks near this company.

starbucks1=api_function('37.3709977,-121.9235529', 'starbucks', 'starbucks_sun1')


In [189]:
print(dist(starbucks1))
print("----------------------------------------------")
print(lat(starbucks1))
print("----------------------------------------------")
print(lng(starbucks1))
print("----------------------------------------------")
print(name(starbucks1))

[555, 110, 550, 1196, 340, 4098, 2273, 3077, 3139, 3739, 1647, 3455, 3024, 2915]
----------------------------------------------
[37.37092547570862, 37.371585845947266, 37.369529724121094, 37.36107375737311, 37.369186, 37.33418343, 37.35060173, 37.345295, 37.395157, 37.40197092074824, 37.385382, 37.341431, 37.398163, 37.38324096]
----------------------------------------------
[-121.91727116354467, -121.92254638671875, -121.92949676513672, -121.92873647557042, -121.92045, -121.9241052, -121.9221276, -121.936371, -121.90523999999999, -121.93991940591599, -121.927919, -121.911642, -121.923706, -121.894416]
----------------------------------------------
['Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks']


In [90]:
dist_starbucks=starbucks1["response"]["groups"][0]["items"][1]["venue"]["location"]["distance"]
lat_starbucks=starbucks1["response"]["groups"][0]["items"][1]["venue"]["location"]["lat"]
lng_starbucks=starbucks1["response"]["groups"][0]["items"][1]["venue"]["location"]["lng"]
name_starbucks=starbucks1["response"]["groups"][0]["items"][1]["venue"]["name"]

print(f"company name: {name_starbucks}, company coordinates: {lat_starbucks},{lng_starbucks}, distance to the offices: {dist_starbucks}")

dist_starbucks=starbucks1["response"]["groups"][0]["items"][4]["venue"]["location"]["distance"]
lat_starbucks=starbucks1["response"]["groups"][0]["items"][4]["venue"]["location"]["lat"]
lng_starbucks=starbucks1["response"]["groups"][0]["items"][4]["venue"]["location"]["lng"]
name_starbucks=starbucks1["response"]["groups"][0]["items"][4]["venue"]["name"]

print(f"company name: {name_starbucks}, company coordinates: {lat_starbucks},{lng_starbucks}, distance to the offices: {dist_starbucks}")


company name: Starbucks, company coordinates: 37.371585845947266,-121.92254638671875, distance to the offices: 110
company name: Starbucks, company coordinates: 37.369186,-121.92045, distance to the offices: 340


There are two near Starbucks, one at 100m and other at 340m.

In [ ]:
# I make a query to the Foursquare API to look for pet hairdressers near this company.

pet1=api_function('37.3709977,-121.9235529', 'Pet Service', 'pet_service_sun1')


In [192]:
print(dist(pet1))
print("----------------------------------------------")
print(lat(pet1))
print("----------------------------------------------")
print(lng(pet1))
print("----------------------------------------------")
print(name(pet1))

[3727, 2702, 4849, 3694, 5100, 2682, 2732, 2839, 2843, 2774, 3377, 3379, 3543, 3424]
----------------------------------------------
[37.339942, 37.3813788, 37.32848854373601, 37.3402263, 37.35306842721221, 37.36269, 37.36273680000001, 37.3796526024096, 37.379786, 37.34972481944317, 37.40111, 37.38021, 37.34043631759206, 37.363336300242906]
----------------------------------------------
[-121.907791, -121.8959331, -121.91154708173961, -121.9079148, -121.97660516837168, -121.95201, -121.9526368, -121.95375089364178, -121.953731, -121.90721511840819, -121.91887, -121.95995900000001, -121.91236495971678, -121.8860647649265]
----------------------------------------------
['PetSmart', 'Banfield Pet Hospital', 'Showtime Pet Grooming', 'Banfield Pet Hospital', 'Petco', 'Ohana Kennels', 'Bay Area K9 Association LLC.', 'A Pet Villa', 'A Pet Villa', 'DogWatch of the Bay Area & Northern California', 'Epic Dog Park', 'Wag Hotels', 'Primal Canine', 'San Jose Animal Hospital']


In [109]:
dist_dog=pet1["response"]["groups"][0]["items"][1]["venue"]["location"]["distance"]
lat_dog=pet1["response"]["groups"][0]["items"][1]["venue"]["location"]["lat"]
lng_dog=pet1["response"]["groups"][0]["items"][1]["venue"]["location"]["lng"]
name_dog=pet1["response"]["groups"][0]["items"][1]["venue"]["name"]

print(f"company name: {name_dog}, company coordinates: {lat_dog},{lng_dog}, distance to the offices: {dist_dog}")


company name: Banfield Pet Hospital, company coordinates: 37.3813788,-121.8959331, distance to the offices: 2702


There is one at 2.7km

-----------------------------

# Second company analysis

In [ ]:
# I make a query to the Foursquare API to look for tech startups near this company.

tech2=api_function('37.37027,-121.924053', 'Tech Startup', 'tech_startups_sj1')


In [195]:
print(dist(tech2))
print("----------------------------------------------")
print(lat(tech2))
print("----------------------------------------------")
print(lng(tech2))
print("----------------------------------------------")
print(name(tech2))

[630, 149, 101, 224, 254, 268, 398, 426, 426, 441, 456, 167, 601, 612]
----------------------------------------------
[37.375874679466456, 37.370972648019674, 37.36947, 37.36957044583222, 37.3695173976203, 37.369302000000005, 37.37009996545508, 37.36805224766053, 37.371297, 37.369042, 37.36753085454393, 37.37114012, 37.36766592556348, 37.368907]
----------------------------------------------
[-121.92306376202592, -121.92549178587224, -121.92351, -121.921677387159, -121.92133577652343, -121.92127099999999, -121.91955220035622, -121.92011894731519, -121.9194084405899, -121.919304, -121.92021896118176, -121.92559373, -121.9180963651028, -121.917341]
----------------------------------------------
['PayPal', 'SourceN HQ', 'FixStream', 'River Meadow Software', 'Robin Systems', 'PlayPhone, Inc.', 'skytree', 'Gaia Online', 'Ocarina Networks', 'SVACPA', 'Ensighten', 'DataTech Labs Data Recovery', 'Nutanix HQ', 'GEO Seminconductor']


In [151]:
dist_tech=tech2["response"]["groups"][0]["items"][1]["venue"]["location"]["distance"]
lat_tech=tech2["response"]["groups"][0]["items"][1]["venue"]["location"]["lat"]
lng_tech=tech2["response"]["groups"][0]["items"][1]["venue"]["location"]["lng"]
name_tech=tech2["response"]["groups"][0]["items"][1]["venue"]["name"]

print(f"company name: {name_tech}, company coordinates: {lat_tech},{lng_tech}, distance to the offices: {dist_tech}")

dist_tech=tech2["response"]["groups"][0]["items"][2]["venue"]["location"]["distance"]
lat_tech=tech2["response"]["groups"][0]["items"][2]["venue"]["location"]["lat"]
lng_tech=tech2["response"]["groups"][0]["items"][2]["venue"]["location"]["lng"]
name_tech=tech2["response"]["groups"][0]["items"][2]["venue"]["name"]

print(f"company name: {name_tech}, company coordinates: {lat_tech},{lng_tech}, distance to the offices: {dist_tech}")

dist_tech=tech2["response"]["groups"][0]["items"][8]["venue"]["location"]["distance"]
lat_tech=tech2["response"]["groups"][0]["items"][8]["venue"]["location"]["lat"]
lng_tech=tech2["response"]["groups"][0]["items"][8]["venue"]["location"]["lng"]
name_tech=tech2["response"]["groups"][0]["items"][8]["venue"]["name"]

print(f"company name: {name_tech}, company coordinates: {lat_tech},{lng_tech}, distance to the offices: {dist_tech}")

company name: SourceN HQ, company coordinates: 37.370972648019674,-121.92549178587224, distance to the offices: 149
company name: FixStream, company coordinates: 37.36947,-121.92351, distance to the offices: 101
company name: DataTech Labs Data Recovery, company coordinates: 37.37114012,-121.92559373, distance to the offices: 167


There are three tech startups near, all of them aroun 150m.

In [ ]:
# I make a query to the Foursquare API to look for design studios near this company.

design2=api_function('37.37027,-121.924053', 'Design Studio', 'design_studio_sj1')


In [198]:
print(dist(design2))
print("----------------------------------------------")
print(lat(design2))
print("----------------------------------------------")
print(lng(design2))
print("----------------------------------------------")
print(name(design2))

[390, 785, 601, 1321, 1377, 1390, 1419, 1420, 1528, 1577, 1647, 1674, 1410, 1845]
----------------------------------------------
[37.37334604119023, 37.3672899, 37.3691219, 37.37102127075195, 37.378426714690484, 37.3707160949707, 37.374528, 37.373359, 37.374013, 37.379857, 37.373688, 37.373843, 37.3657117, 37.365846999999995]
----------------------------------------------
[-121.92193508148193, -121.916, -121.917411, -121.90914154052734, -121.91234536963067, -121.9083480834961, -121.908934, -121.90847, -121.907427, -121.910925, -121.905933, -121.905661, -121.9091752, -121.94416000000001]
----------------------------------------------
['Kiskon Technologies', 'Watry Design', 'Konica Minolta Business Solutions', 'Screened Magic', 'Home Expo &  Design', 'Woodenbridge', 'Summit Cabinet', 'The Ceramic Corporation', 'Atlas Glass', 'Bay Stone', 'Granite & Stone Expo', 'Zeus Tile', 'Signarama San Jose', 'All Architectural Stone']


In [129]:
dist_design=design2["response"]["groups"][0]["items"][1]["venue"]["location"]["distance"]
lat_design=design2["response"]["groups"][0]["items"][1]["venue"]["location"]["lat"]
lng_design=design2["response"]["groups"][0]["items"][1]["venue"]["location"]["lng"]
name_design=design2["response"]["groups"][0]["items"][1]["venue"]["name"]

print(f"company name: {name_design}, company coordinates: {lat_design},{lng_design}, distance to the offices: {dist_design}")

company name: Watry Design, company coordinates: 37.3672899,-121.916, distance to the offices: 785


There is one design studio near at 785m

In [ ]:
# I make a query to the Foursquare API to look for schools near this company.

sch2=api_function('37.37027,-121.924053', 'Elementary School', 'elementary_sch_sj1')


In [201]:
print(dist(sch2))
print("----------------------------------------------")
print(lat(sch2))
print("----------------------------------------------")
print(lng(sch2))
print("----------------------------------------------")
print(name(sch2))

[1262, 2857, 2929, 3259, 3288, 3368, 3406, 3048, 3635, 3847, 4044, 3718, 4422, 4449]
----------------------------------------------
[37.36330687882652, 37.388926007147695, 37.38955552112065, 37.39309384415278, 37.358551025390625, 37.343393216725616, 37.35738251403415, 37.347336, 37.348379, 37.350991, 37.38721347682061, 37.398261959325076, 37.333647875243265, 37.3303300740432]
----------------------------------------------
[-121.9127954630632, -121.90186142921448, -121.94658055857437, -121.94713050142707, -121.95817565917969, -121.94153776982016, -121.9589768845584, -121.942885, -121.893555, -121.88796799999999, -121.88360782691444, -121.94699718220039, -121.94342527769899, -121.92598071783651]
----------------------------------------------
['Bachrodt Elementary School', 'Orchard Elementary School', 'Montague Elementary School', 'Don Callejon School', 'Jano Hindi Bee', 'Washington Open Elementary', 'Scott Lane Elementary School', "St Clare's Catholic School", 'Suzume No Gakko', 'Grant E

In [132]:
dist_school=sch2["response"]["groups"][0]["items"][1]["venue"]["location"]["distance"]
lat_school=sch2["response"]["groups"][0]["items"][1]["venue"]["location"]["lat"]
lng_school=sch2["response"]["groups"][0]["items"][1]["venue"]["location"]["lng"]
name_school=sch2["response"]["groups"][0]["items"][1]["venue"]["name"]

print(f"company name: {name_school}, company coordinates: {lat_school},{lng_school}, distance to the offices: {dist_school}")

company name: Orchard Elementary School, company coordinates: 37.388926007147695,-121.90186142921448, distance to the offices: 2857


The nearest school is at 2.8km

In [ ]:
# I make a query to the Foursquare API to look for nighlife spots near this company.

night2=api_function('37.37027,-121.924053', 'Nightlife Spot', 'nightlife_spot_sj1')


In [204]:
print(dist(night2))
print("----------------------------------------------")
print(lat(night2))
print("----------------------------------------------")
print(lng(night2))
print("----------------------------------------------")
print(name(night2))

[1730, 2801, 398, 1603, 514, 206, 222, 460, 478, 482, 584, 613, 647, 718]
----------------------------------------------
[37.36086599567488, 37.36132088609424, 37.37124135626228, 37.36381793355914, 37.365813193961756, 37.37152348661807, 37.371532546100354, 37.36690004129253, 37.366398, 37.369526, 37.375498869419765, 37.375773, 37.37026696954015, 37.36545102537579]
----------------------------------------------
[-121.90847504592831, -121.95364903471561, -121.91972331072563, -121.90784481894752, -121.9256078829507, -121.92232810486176, -121.9220969824811, -121.92706413888456, -121.92641, -121.929421, -121.92348539829254, -121.923609, -121.93136993764425, -121.9186465104381]
----------------------------------------------
['The Doghouse Sports Lounge', 'Golden State Brewery', 'Zone 8 Sports Bar & Grill', '4th St Bowl Lounge', 'Santa Cruz Wine Bar', 'Doubletree Pool', 'Fortyeight at DoubleTree', 'shark cage', 'T Baggers', 'San Jose Beer Union', 'GSPS Wine Friday', 'Please Be Thirsty', "Chia

In [156]:
dist_party=night2["response"]["groups"][0]["items"][2]["venue"]["location"]["distance"]
lat_party=night2["response"]["groups"][0]["items"][2]["venue"]["location"]["lat"]
lng_party=night2["response"]["groups"][0]["items"][2]["venue"]["location"]["lng"]
name_party=night2["response"]["groups"][0]["items"][2]["venue"]["name"]

print(f"company name: {name_party}, company coordinates: {lat_party},{lng_party}, distance to the offices: {dist_party}")

dist_party=night2["response"]["groups"][0]["items"][9]["venue"]["location"]["distance"]
lat_party=night2["response"]["groups"][0]["items"][9]["venue"]["location"]["lat"]
lng_party=night2["response"]["groups"][0]["items"][9]["venue"]["location"]["lng"]
name_party=night2["response"]["groups"][0]["items"][9]["venue"]["name"]

print(f"company name: {name_party}, company coordinates: {lat_party},{lng_party}, distance to the offices: {dist_party}")

dist_party=night2["response"]["groups"][0]["items"][5]["venue"]["location"]["distance"]
lat_party=night2["response"]["groups"][0]["items"][5]["venue"]["location"]["lat"]
lng_party=night2["response"]["groups"][0]["items"][5]["venue"]["location"]["lng"]
name_party=night2["response"]["groups"][0]["items"][5]["venue"]["name"]

print(f"company name: {name_party}, company coordinates: {lat_party},{lng_party}, distance to the offices: {dist_party}")

company name: Zone 8 Sports Bar & Grill, company coordinates: 37.37124135626228,-121.91972331072563, distance to the offices: 398
company name: 2050 Lobby Lounge & Bar, company coordinates: 37.371769553739405,-121.92226542378253, distance to the offices: 229
company name: Doubletree Pool, company coordinates: 37.37152348661807,-121.92232810486176, distance to the offices: 206


There are three noghlife spots near around 200m and 300m.

In [ ]:
# I make a query to the Foursquare API to look for vegetarian restaurants near this company.

vegan2=api_function('37.37027,-121.924053', 'Vegan Restaurant', 'veg_rest_sj1')


In [207]:
print(dist(vegan2))
print("----------------------------------------------")
print(lat(vegan2))
print("----------------------------------------------")
print(lng(vegan2))
print("----------------------------------------------")
print(name(vegan2))

[2961, 4902, 4818, 5106, 3301, 5001, 5562, 5229, 2210, 2535, 2439, 3320, 3671, 3863]
----------------------------------------------
[37.34460383642609, 37.33547578302974, 37.336852209398536, 37.341433051588915, 37.343718446032305, 37.33993504196706, 37.330296164668816, 37.32614245482791, 37.385706, 37.35659, 37.3607916, 37.34049606323242, 37.33930772, 37.338702000000005]
----------------------------------------------
[-121.93286117841694, -121.89009385118939, -121.88943603809092, -121.87917600579642, -121.9406861466985, -121.88235410455921, -121.88632717974279, -121.94432964084655, -121.908328, -121.94696, -121.8991962, -121.92169952392578, -121.93835143, -121.942205]
----------------------------------------------
["Ike's Sandwiches", 'Good Karma', "Ike's Sandwiches", 'Vegetarian House', "deedee's", 'Tofoo Com Chay', 'Vitamina Juices & Blends', 'Loving Hut', 'Vegefarm', 'The Happy Hooligans', "Brother Baby's BBQ", 'Queen Of Sheba', 'Raw Vittles', 'Clean Eating with Katie']


In [158]:
dist_vegan=vegan2["response"]["groups"][0]["items"][8]["venue"]["location"]["distance"]
lat_vegan=vegan2["response"]["groups"][0]["items"][8]["venue"]["location"]["lat"]
lng_vegan=vegan2["response"]["groups"][0]["items"][8]["venue"]["location"]["lng"]
name_vegan=vegan2["response"]["groups"][0]["items"][8]["venue"]["name"]

print(f"company name: {name_vegan}, company coordinates: {lat_vegan},{lng_vegan}, distance to the offices: {dist_vegan}")

company name: Brother Baby's BBQ, company coordinates: 37.3607916,-121.8991962, distance to the offices: 2439


The nearest vegan restaurant is at 1.4km

In [ ]:
# I make a query to the Foursquare API to look for basketball courts near this company.

basket2=api_function('37.37027,-121.924053', 'Basketball Court', 'basketball_court_sj1')


In [210]:
print(dist(basket2))
print("----------------------------------------------")
print(lat(basket2))
print("----------------------------------------------")
print(lng(basket2))
print("----------------------------------------------")
print(name(basket2))

[3633, 754, 1013, 1103, 1120, 2783, 2999, 3607, 3933, 4058, 4182, 4715, 4403, 4880]
----------------------------------------------
[37.33779578622328, 37.37696075439453, 37.36441421508789, 37.364402, 37.37117385864258, 37.3881680172003, 37.377344606195855, 37.39918075740896, 37.37372, 37.3753662109375, 37.33270914406301, 37.40387815016547, 37.330723171109355, 37.3980712890625]
----------------------------------------------
[-121.91989631992244, -121.92266082763672, -121.91527557373047, -121.913997, -121.91143798828125, -121.94602505794903, -121.9567734963286, -121.94248645305073, -121.968306, -121.96947479248047, -121.9253028797591, -121.9565124356151, -121.92286373936807, -121.88139343261719]
----------------------------------------------
['Central YMCA', 'eBay Inc. Basketball Court', 'Brocade Courts', 'Skyport!', 'Basketball Gym', 'Montague Park', '3 Ring', 'mansion grove basketball court', "Rucker's Basketball Court", 'Podium Golden State Warriors Basketball Court', 'Hoover Gym', 'L

In [142]:
dist_basket=basket2["response"]["groups"][0]["items"][1]["venue"]["location"]["distance"]
lat_basket=basket2["response"]["groups"][0]["items"][1]["venue"]["location"]["lat"]
lng_basket=basket2["response"]["groups"][0]["items"][1]["venue"]["location"]["lng"]
name_basket=basket2["response"]["groups"][0]["items"][1]["venue"]["name"]

print(f"company name: {name_basket}, company coordinates: {lat_basket},{lng_basket}, distance to the offices: {dist_basket}")

company name: eBay Inc. Basketball Court, company coordinates: 37.37696075439453,-121.92266082763672, distance to the offices: 754


The nearest basketball court is at 754m

In [ ]:
# I make a query to the Foursquare API to look for Starbucks near this company.

starbucks2=api_function('37.37027,-121.924053', 'starbucks', 'starbucks_sj1')


In [213]:
print(dist(starbucks2))
print("----------------------------------------------")
print(lat(starbucks2))
print("----------------------------------------------")
print(lng(starbucks2))
print("----------------------------------------------")
print(name(starbucks2))

[604, 198, 488, 1104, 340, 4017, 2196, 2986, 3392, 3231, 3797, 1716, 3105, 607]
----------------------------------------------
[37.37092547570862, 37.371585845947266, 37.369529724121094, 37.36107375737311, 37.369186, 37.33418343, 37.35060173, 37.345295, 37.341431, 37.395157, 37.40197092074824, 37.385382, 37.398163, 37.36482957582107]
----------------------------------------------
[-121.91727116354467, -121.92254638671875, -121.92949676513672, -121.92873647557042, -121.92045, -121.9241052, -121.9221276, -121.936371, -121.911642, -121.90523999999999, -121.93991940591599, -121.927919, -121.923706, -121.92356576168726]
----------------------------------------------
['Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks']


In [161]:
dist_starbucks=starbucks2["response"]["groups"][0]["items"][1]["venue"]["location"]["distance"]
lat_starbucks=starbucks2["response"]["groups"][0]["items"][1]["venue"]["location"]["lat"]
lng_starbucks=starbucks2["response"]["groups"][0]["items"][1]["venue"]["location"]["lng"]
name_starbucks=starbucks2["response"]["groups"][0]["items"][1]["venue"]["name"]

print(f"company name: {name_starbucks}, company coordinates: {lat_starbucks},{lng_starbucks}, distance to the offices: {dist_starbucks}")

dist_starbucks=starbucks2["response"]["groups"][0]["items"][4]["venue"]["location"]["distance"]
lat_starbucks=starbucks2["response"]["groups"][0]["items"][4]["venue"]["location"]["lat"]
lng_starbucks=starbucks2["response"]["groups"][0]["items"][4]["venue"]["location"]["lng"]
name_starbucks=starbucks2["response"]["groups"][0]["items"][4]["venue"]["name"]

print(f"company name: {name_starbucks}, company coordinates: {lat_starbucks},{lng_starbucks}, distance to the offices: {dist_starbucks}")

company name: Starbucks, company coordinates: 37.371585845947266,-121.92254638671875, distance to the offices: 198
company name: Starbucks, company coordinates: 37.369186,-121.92045, distance to the offices: 340


There are two Starbucks near

In [ ]:
# I make a query to the Foursquare API to look for pet hairdressers near this company.

pet2=api_function('37.37027,-121.924053', 'Pet Service', 'pet_service_sj1')


In [217]:
print(dist(pet2))
print("----------------------------------------------")
print(lat(pet2))
print("----------------------------------------------")
print(lng(pet2))
print("----------------------------------------------")
print(name(pet2))

[3669, 2777, 4780, 3636, 5028, 2613, 2664, 2827, 2831, 2729, 3363, 3463, 3478, 3592]
----------------------------------------------
[37.339942, 37.3813788, 37.32848854373601, 37.3402263, 37.35306842721221, 37.36269, 37.36273680000001, 37.3796526024096, 37.379786, 37.34972481944317, 37.38021, 37.40111, 37.34043631759206, 37.343559452656756]
----------------------------------------------
[-121.907791, -121.8959331, -121.91154708173961, -121.9079148, -121.97660516837168, -121.95201, -121.9526368, -121.95375089364178, -121.953731, -121.90721511840819, -121.95995900000001, -121.91887, -121.91236495971678, -121.94683830386161]
----------------------------------------------
['PetSmart', 'Banfield Pet Hospital', 'Showtime Pet Grooming', 'Banfield Pet Hospital', 'Petco', 'Ohana Kennels', 'Bay Area K9 Association LLC.', 'A Pet Villa', 'A Pet Villa', 'DogWatch of the Bay Area & Northern California', 'Wag Hotels', 'Epic Dog Park', 'Primal Canine', "Juan's Groom Room"]


In [162]:
dist_dog=pet2["response"]["groups"][0]["items"][5]["venue"]["location"]["distance"]
lat_dog=pet2["response"]["groups"][0]["items"][5]["venue"]["location"]["lat"]
lng_dog=pet2["response"]["groups"][0]["items"][5]["venue"]["location"]["lng"]
name_dog=pet2["response"]["groups"][0]["items"][5]["venue"]["name"]

print(f"company name: {name_dog}, company coordinates: {lat_dog},{lng_dog}, distance to the offices: {dist_dog}")

company name: Ohana Kennels, company coordinates: 37.36269,-121.95201, distance to the offices: 2613


Near the offices there is a pet hairdresser at 2.6km

-----------------------------

# Third company analysis

In [ ]:
# I make a query to the Foursquare API to look for tech startups near this company.

tech3=api_function('37.371088,-121.925355', 'Tech Startup', 'tech_startups_sj2')


In [250]:
print(dist(tech3))
print("----------------------------------------------")
print(lat(tech3))
print("----------------------------------------------")
print(lng(tech3))
print("----------------------------------------------")
print(name(tech3))

[570, 17, 243, 366, 396, 412, 530, 573, 21, 602, 702, 746, 749, 758]
----------------------------------------------
[37.375874679466456, 37.370972648019674, 37.36947, 37.36957044583222, 37.3695173976203, 37.369302000000005, 37.369813, 37.36805224766053, 37.37114012, 37.36753085454393, 37.377064, 37.36766592556348, 37.368907, 37.367859]
----------------------------------------------
[-121.92306376202592, -121.92549178587224, -121.92351, -121.921677387159, -121.92133577652343, -121.92127099999999, -121.919579, -121.92011894731519, -121.92559373, -121.92021896118176, -121.922804, -121.9180963651028, -121.917341, -121.917805]
----------------------------------------------
['PayPal', 'SourceN HQ', 'FixStream', 'River Meadow Software', 'Robin Systems', 'PlayPhone, Inc.', 'Fohboh Headquarters', 'Gaia Online', 'DataTech Labs Data Recovery', 'Ensighten', 'PayPal IT Helpdesk', 'Nutanix HQ', 'GEO Seminconductor', 'Usens']


In [12]:
dist_tech=tech3["response"]["groups"][0]["items"][1]["venue"]["location"]["distance"]
lat_tech=tech3["response"]["groups"][0]["items"][1]["venue"]["location"]["lat"]
lng_tech=tech3["response"]["groups"][0]["items"][1]["venue"]["location"]["lng"]
name_tech=tech3["response"]["groups"][0]["items"][1]["venue"]["name"]

print(f"company name: {name_tech}, company coordinates: {lat_tech},{lng_tech}, distance to the offices: {dist_tech}")

dist_tech=tech3["response"]["groups"][0]["items"][8]["venue"]["location"]["distance"]
lat_tech=tech3["response"]["groups"][0]["items"][8]["venue"]["location"]["lat"]
lng_tech=tech3["response"]["groups"][0]["items"][8]["venue"]["location"]["lng"]
name_tech=tech3["response"]["groups"][0]["items"][8]["venue"]["name"]

print(f"company name: {name_tech}, company coordinates: {lat_tech},{lng_tech}, distance to the offices: {dist_tech}")

company name: SourceN HQ, company coordinates: 37.370972648019674,-121.92549178587224, distance to the offices: 17
company name: DataTech Labs Data Recovery, company coordinates: 37.37114012,-121.92559373, distance to the offices: 21


There are two tech startups very close to the company

In [ ]:
# I make a query to the Foursquare API to look for design studios near this company.

design3=api_function('37.371088,-121.925355', 'Design Studio', 'design_studio_sj2')


In [249]:
print(dist(design3))
print("----------------------------------------------")
print(lat(design3))
print("----------------------------------------------")
print(lng(design3))
print("----------------------------------------------")
print(name(design3))

[393, 929, 736, 1411, 1505, 1606, 1619, 1742, 1763, 1854, 1551, 1744, 1826, 2305]
----------------------------------------------
[37.37334604119023, 37.3672899, 37.3691219, 37.378426714690484, 37.3707160949707, 37.379857, 37.374013, 37.373688, 37.365846999999995, 37.383968353271484, 37.3657117, 37.3656546, 37.38424, 37.38526153564453]
----------------------------------------------
[-121.92193508148193, -121.916, -121.917411, -121.91234536963067, -121.9083480834961, -121.910925, -121.907427, -121.905933, -121.94416000000001, -121.91206359863281, -121.9091752, -121.9068635, -121.91301, -121.9063491821289]
----------------------------------------------
['Kiskon Technologies', 'Watry Design', 'Konica Minolta Business Solutions', 'Home Expo &  Design', 'Woodenbridge', 'Bay Stone', 'Atlas Glass', 'Granite & Stone Expo', 'All Architectural Stone', 'Emser Tile&Natural Stone', 'Signarama San Jose', 'Speck Design', '104 Octane', 'Bath Solutions by Mike Counsil']


In [16]:
dist_design=design3["response"]["groups"][0]["items"][0]["venue"]["location"]["distance"]
lat_design=design3["response"]["groups"][0]["items"][0]["venue"]["location"]["lat"]
lng_design=design3["response"]["groups"][0]["items"][0]["venue"]["location"]["lng"]
name_design=design3["response"]["groups"][0]["items"][0]["venue"]["name"]

print(f"company name: {name_design}, company coordinates: {lat_design},{lng_design}, distance to the offices: {dist_design}")

company name: Kiskon Technologies, company coordinates: 37.37334604119023,-121.92193508148193, distance to the offices: 393


There is one design studio near around 400m

In [ ]:
# I make a query to the Foursquare API to look for schools near this company.

sch3=api_function('37.371088,-121.925355', 'Elementary School', 'elementary_sch_sj2')


In [248]:
print(dist(sch3))
print("----------------------------------------------")
print(lat(sch3))
print("----------------------------------------------")
print(lng(sch3))
print("----------------------------------------------")
print(name(sch3))

[1408, 2784, 2874, 3343, 3399, 3065, 3782, 3579, 3993, 4106, 4464, 4537, 4577, 4650]
----------------------------------------------
[37.36330687882652, 37.38955552112065, 37.388926007147695, 37.35738251403415, 37.343393216725616, 37.347336, 37.348379, 37.398261959325076, 37.350991, 37.38721347682061, 37.333647875243265, 37.3303300740432, 37.40608596801758, 37.343032172509545]
----------------------------------------------
[-121.9127954630632, -121.94658055857437, -121.90186142921448, -121.9589768845584, -121.94153776982016, -121.942885, -121.893555, -121.94699718220039, -121.88796799999999, -121.88360782691444, -121.94342527769899, -121.92598071783651, -121.898193359375, -121.96429400003163]
----------------------------------------------
['Bachrodt Elementary School', 'Montague Elementary School', 'Orchard Elementary School', 'Scott Lane Elementary School', 'Washington Open Elementary', "St Clare's Catholic School", 'Suzume No Gakko', 'Don Callejon School', 'Grant Elementary School', '

In [19]:
dist_school=sch3["response"]["groups"][0]["items"][0]["venue"]["location"]["distance"]
lat_school=sch3["response"]["groups"][0]["items"][0]["venue"]["location"]["lat"]
lng_school=sch3["response"]["groups"][0]["items"][0]["venue"]["location"]["lng"]
name_school=sch3["response"]["groups"][0]["items"][0]["venue"]["name"]

print(f"company name: {name_school}, company coordinates: {lat_school},{lng_school}, distance to the offices: {dist_school}")

company name: Bachrodt Elementary School, company coordinates: 37.36330687882652,-121.9127954630632, distance to the offices: 1408


The nearest school is at 1.4km

In [ ]:
# I make a query to the Foursquare API to look for nighlife spots near this company.

night3=api_function('37.371088,-121.925355', 'Nightlife Spot', 'nightlife_spot_sj2')


In [247]:
print(dist(night3))
print("----------------------------------------------")
print(lat(night3))
print("----------------------------------------------")
print(lng(night3))
print("----------------------------------------------")
print(name(night3))

[1877, 2729, 498, 1747, 587, 272, 490, 509, 283, 2448, 486, 476, 1467, 2159]
----------------------------------------------
[37.36086599567488, 37.36132088609424, 37.37124135626228, 37.36381793355914, 37.365813193961756, 37.37152348661807, 37.36690004129253, 37.37082598391839, 37.371769553739405, 37.35856138262397, 37.36684205185309, 37.36699705883182, 37.37458419799805, 37.3517028899461]
----------------------------------------------
[-121.90847504592831, -121.95364903471561, -121.91972331072563, -121.90784481894752, -121.9256078829507, -121.92232810486176, -121.92706413888456, -121.93110972058392, -121.92226542378253, -121.90260409915966, -121.92666041405536, -121.92694046682892, -121.90935516357422, -121.92446824885526]
----------------------------------------------
['The Doghouse Sports Lounge', 'Golden State Brewery', 'Zone 8 Sports Bar & Grill', '4th St Bowl Lounge', 'Santa Cruz Wine Bar', 'Doubletree Pool', 'shark cage', 'Access Point', '2050 Lobby Lounge & Bar', 'Hyde Park Loun

In [26]:

dist_party=night3["response"]["groups"][0]["items"][8]["venue"]["location"]["distance"]
lat_party=night3["response"]["groups"][0]["items"][8]["venue"]["location"]["lat"]
lng_party=night3["response"]["groups"][0]["items"][8]["venue"]["location"]["lng"]
name_party=night3["response"]["groups"][0]["items"][8]["venue"]["name"]

print(f"company name: {name_party}, company coordinates: {lat_party},{lng_party}, distance to the offices: {dist_party}")

dist_party=night3["response"]["groups"][0]["items"][5]["venue"]["location"]["distance"]
lat_party=night3["response"]["groups"][0]["items"][5]["venue"]["location"]["lat"]
lng_party=night3["response"]["groups"][0]["items"][5]["venue"]["location"]["lng"]
name_party=night3["response"]["groups"][0]["items"][5]["venue"]["name"]

print(f"company name: {name_party}, company coordinates: {lat_party},{lng_party}, distance to the offices: {dist_party}")



company name: 2050 Lobby Lounge & Bar, company coordinates: 37.371769553739405,-121.92226542378253, distance to the offices: 283
company name: Doubletree Pool, company coordinates: 37.37152348661807,-121.92232810486176, distance to the offices: 272


The nearest nighlife spots are at 300m approximately

In [ ]:
# I make a query to the Foursquare API to look for vegetarian restaurants near this company.

vegan3=api_function('37.371088,-121.925355', 'Vegan Restaurant', 'veg_rest_sj2')


In [246]:
print(dist(vegan3))
print("----------------------------------------------")
print(lat(vegan3))
print("----------------------------------------------")
print(lng(vegan3))
print("----------------------------------------------")
print(name(vegan3))

[3022, 3335, 5277, 2217, 2501, 2582, 3420, 3719, 3901, 4114, 4257, 5065, 4786, 5600]
----------------------------------------------
[37.34460383642609, 37.343718446032305, 37.32614245482791, 37.385706, 37.35659, 37.3607916, 37.34049606323242, 37.33930772, 37.338702000000005, 37.404856, 37.396342, 37.35198682571322, 37.41086084497713, 37.42087245286653]
----------------------------------------------
[-121.93286117841694, -121.9406861466985, -121.94432964084655, -121.908328, -121.94696, -121.8991962, -121.92169952392578, -121.93835143, -121.942205, -121.906444, -121.88920700000001, -121.97731920128297, -121.94593038741806, -121.91625220186718]
----------------------------------------------
["Ike's Sandwiches", "deedee's", 'Loving Hut', 'Vegefarm', 'The Happy Hooligans', "Brother Baby's BBQ", 'Queen Of Sheba', 'Raw Vittles', 'Clean Eating with Katie', 'Tasty Vegetarian Cuisine', 'SJ Vegetarian', 'Peacock Indian Cuisine', "Sri's Kitchen", 'Loving Hut']


In [31]:
dist_vegan=vegan3["response"]["groups"][0]["items"][3]["venue"]["location"]["distance"]
lat_vegan=vegan3["response"]["groups"][0]["items"][3]["venue"]["location"]["lat"]
lng_vegan=vegan3["response"]["groups"][0]["items"][3]["venue"]["location"]["lng"]
name_vegan=vegan3["response"]["groups"][0]["items"][3]["venue"]["name"]

print(f"company name: {name_vegan}, company coordinates: {lat_vegan},{lng_vegan}, distance to the offices: {dist_vegan}")

company name: Brother Baby's BBQ, company coordinates: 37.3607916,-121.8991962, distance to the offices: 2582


The nearest one is at 2.5km

In [ ]:
# I make a query to the Foursquare API to look for basketball courts near this company.

basket3=api_function('37.371088,-121.925355', 'Basketball Court', 'basketball_court_sj2')


In [242]:
print(dist(basket3))
print("----------------------------------------------")
print(lat(basket3))
print("----------------------------------------------")
print(lng(basket3))
print("----------------------------------------------")
print(name(basket3))

[3737, 695, 1160, 1231, 1250, 2637, 2865, 3475, 3810, 4573, 5035, 4136, 5514, 4619]
----------------------------------------------
[37.33779578622328, 37.37696075439453, 37.36441421508789, 37.37117385864258, 37.364402, 37.3881680172003, 37.377344606195855, 37.39918075740896, 37.37372, 37.40387815016547, 37.41575241088867, 37.393019101339256, 37.35904935160653, 37.356725437277845]
----------------------------------------------
[-121.91989631992244, -121.92266082763672, -121.91527557373047, -121.91143798828125, -121.913997, -121.94602505794903, -121.9567734963286, -121.94248645305073, -121.968306, -121.9565124356151, -121.91634368896484, -121.96311097120844, -121.98581956228077, -121.87637085671278]
----------------------------------------------
['Central YMCA', 'eBay Inc. Basketball Court', 'Brocade Courts', 'Basketball Gym', 'Skyport!', 'Montague Park', '3 Ring', 'mansion grove basketball court', "Rucker's Basketball Court", 'Lick Mill Park', 'Cisco - Basketball Court', 'Agnew Park', '

In [34]:
dist_basket=basket3["response"]["groups"][0]["items"][1]["venue"]["location"]["distance"]
lat_basket=basket3["response"]["groups"][0]["items"][1]["venue"]["location"]["lat"]
lng_basket=basket3["response"]["groups"][0]["items"][1]["venue"]["location"]["lng"]
name_basket=basket3["response"]["groups"][0]["items"][1]["venue"]["name"]

print(f"company name: {name_basket}, company coordinates: {lat_basket},{lng_basket}, distance to the offices: {dist_basket}")

company name: eBay Inc. Basketball Court, company coordinates: 37.37696075439453,-121.92266082763672, distance to the offices: 695


There is one basketball court at 700m

In [ ]:
# I make a query to the Foursquare API to look for Starbucks near this company.

starbucks3=api_function('37.371088,-121.925355', 'starbucks', 'starbucks_sj2')


In [241]:
print(dist(starbucks3))
print("----------------------------------------------")
print(lat(starbucks3))
print("----------------------------------------------")
print(lng(starbucks3))
print("----------------------------------------------")
print(name(starbucks3))

[715, 254, 405, 1154, 482, 2298, 3032, 4109, 1607, 3216, 3017, 3517, 3671, 3413]
----------------------------------------------
[37.37092547570862, 37.371585845947266, 37.369529724121094, 37.36107375737311, 37.369186, 37.35060173, 37.345295, 37.33418343, 37.385382, 37.395157, 37.398163, 37.341431, 37.40197092074824, 37.39293751646301]
----------------------------------------------
[-121.91727116354467, -121.92254638671875, -121.92949676513672, -121.92873647557042, -121.92045, -121.9221276, -121.936371, -121.9241052, -121.927919, -121.90523999999999, -121.923706, -121.911642, -121.93991940591599, -121.95243536434629]
----------------------------------------------
['Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks']


In [38]:

dist_starbucks=starbucks3["response"]["groups"][0]["items"][1]["venue"]["location"]["distance"]
lat_starbucks=starbucks3["response"]["groups"][0]["items"][1]["venue"]["location"]["lat"]
lng_starbucks=starbucks3["response"]["groups"][0]["items"][1]["venue"]["location"]["lng"]
name_starbucks=starbucks3["response"]["groups"][0]["items"][1]["venue"]["name"]

print(f"company name: {name_starbucks}, company coordinates: {lat_starbucks},{lng_starbucks}, distance to the offices: {dist_starbucks}")

company name: Starbucks, company coordinates: 37.371585845947266,-121.92254638671875, distance to the offices: 254


Near the company there is a Starbucks at 254m

In [ ]:
# I make a query to the Foursquare API to look for pet hairdressers near this company.

pet3=api_function('37.371088,-121.925355', 'Pet Service', 'pet_service_sj2')


In [240]:
print(dist(pet3))
print("----------------------------------------------")
print(lat(pet3))
print("----------------------------------------------")
print(lng(pet3))
print("----------------------------------------------")
print(name(pet3))

[3799, 2843, 4897, 3766, 4958, 2536, 2586, 2686, 2869, 3225, 3390, 3600, 3606, 3622]
----------------------------------------------
[37.339942, 37.3813788, 37.32848854373601, 37.3402263, 37.35306842721221, 37.36269, 37.36273680000001, 37.3796526024096, 37.34972481944317, 37.38021, 37.40111, 37.34043631759206, 37.343559452656756, 37.343485]
----------------------------------------------
[-121.907791, -121.8959331, -121.91154708173961, -121.9079148, -121.97660516837168, -121.95201, -121.9526368, -121.95375089364178, -121.90721511840819, -121.95995900000001, -121.91887, -121.91236495971678, -121.94683830386161, -121.947041]
----------------------------------------------
['PetSmart', 'Banfield Pet Hospital', 'Showtime Pet Grooming', 'Banfield Pet Hospital', 'Petco', 'Ohana Kennels', 'Bay Area K9 Association LLC.', 'A Pet Villa', 'DogWatch of the Bay Area & Northern California', 'Wag Hotels', 'Epic Dog Park', 'Primal Canine', "Juan's Groom Room", 'The Tails In The City']


In [43]:
dist_dog=pet3["response"]["groups"][0]["items"][6]["venue"]["location"]["distance"]
lat_dog=pet3["response"]["groups"][0]["items"][6]["venue"]["location"]["lat"]
lng_dog=pet3["response"]["groups"][0]["items"][6]["venue"]["location"]["lng"]
name_dog=pet3["response"]["groups"][0]["items"][6]["venue"]["name"]

print(f"company name: {name_dog}, company coordinates: {lat_dog},{lng_dog}, distance to the offices: {dist_dog}")

company name: Bay Area K9 Association LLC., company coordinates: 37.36273680000001,-121.9526368, distance to the offices: 2586


The pet hairdresser is at 2.5km from the company

-------

# Fourth company analysis

In [52]:
# I make a query to the Foursquare API to look for tech startups near this company.

tech4=api_function('37.3721771, -121.9250398', 'Tech Startup', 'tech_startups_sj3')


In [64]:
print(dist(tech4))
print("----------------------------------------------")
print(lat(tech4))
print("----------------------------------------------")
print(lng(tech4))
print("----------------------------------------------")
print(name(tech4))

[447, 139, 62, 330, 415, 441, 462, 507, 537, 550, 578, 125, 615, 632]
----------------------------------------------
[37.375874679466456, 37.370972648019674, 37.372206, 37.36947, 37.36957044583222, 37.3695173976203, 37.369302000000005, 37.371297, 37.37009996545508, 37.369813, 37.377064, 37.37114012, 37.369042, 37.36805224766053]
----------------------------------------------
[-121.92306376202592, -121.92549178587224, -121.925749, -121.92351, -121.921677387159, -121.92133577652343, -121.92127099999999, -121.9194084405899, -121.91955220035622, -121.919579, -121.922804, -121.92559373, -121.919304, -121.92011894731519]
----------------------------------------------
['PayPal', 'SourceN HQ', 'Ooyala', 'FixStream', 'River Meadow Software', 'Robin Systems', 'PlayPhone, Inc.', 'Ocarina Networks', 'skytree', 'Fohboh Headquarters', 'PayPal IT Helpdesk', 'DataTech Labs Data Recovery', 'SVACPA', 'Gaia Online']


In [66]:
dist_tech=tech4["response"]["groups"][0]["items"][2]["venue"]["location"]["distance"]
lat_tech=tech4["response"]["groups"][0]["items"][2]["venue"]["location"]["lat"]
lng_tech=tech4["response"]["groups"][0]["items"][2]["venue"]["location"]["lng"]
name_tech=tech4["response"]["groups"][0]["items"][2]["venue"]["name"]

print(f"company name: {name_tech}, company coordinates: {lat_tech},{lng_tech}, distance to the offices: {dist_tech}")

dist_tech=tech4["response"]["groups"][0]["items"][1]["venue"]["location"]["distance"]
lat_tech=tech4["response"]["groups"][0]["items"][1]["venue"]["location"]["lat"]
lng_tech=tech4["response"]["groups"][0]["items"][1]["venue"]["location"]["lng"]
name_tech=tech4["response"]["groups"][0]["items"][1]["venue"]["name"]

print(f"company name: {name_tech}, company coordinates: {lat_tech},{lng_tech}, distance to the offices: {dist_tech}")

dist_tech=tech4["response"]["groups"][0]["items"][11]["venue"]["location"]["distance"]
lat_tech=tech4["response"]["groups"][0]["items"][11]["venue"]["location"]["lat"]
lng_tech=tech4["response"]["groups"][0]["items"][11]["venue"]["location"]["lng"]
name_tech=tech4["response"]["groups"][0]["items"][11]["venue"]["name"]

print(f"company name: {name_tech}, company coordinates: {lat_tech},{lng_tech}, distance to the offices: {dist_tech}")

company name: Ooyala, company coordinates: 37.372206,-121.925749, distance to the offices: 62
company name: SourceN HQ, company coordinates: 37.370972648019674,-121.92549178587224, distance to the offices: 139
company name: DataTech Labs Data Recovery, company coordinates: 37.37114012,-121.92559373, distance to the offices: 125


There are three tech companies very close

In [67]:
# I make a query to the Foursquare API to look for design studios near this company.

design4=api_function('37.3721771, -121.9250398', 'Design Studio', 'design_studio_sj3')


In [68]:
print(dist(design4))
print("----------------------------------------------")
print(lat(design4))
print("----------------------------------------------")
print(lng(design4))
print("----------------------------------------------")
print(name(design4))

[303, 967, 755, 1321, 1412, 1448, 1471, 1485, 1513, 1571, 1698, 1724, 1743, 1832]
----------------------------------------------
[37.37334604119023, 37.3672899, 37.3691219, 37.378426714690484, 37.37102127075195, 37.374528, 37.373359, 37.3707160949707, 37.379857, 37.374013, 37.373688, 37.373843, 37.383968353271484, 37.365846999999995]
----------------------------------------------
[-121.92193508148193, -121.916, -121.917411, -121.91234536963067, -121.90914154052734, -121.908934, -121.90847, -121.9083480834961, -121.910925, -121.907427, -121.905933, -121.905661, -121.91206359863281, -121.94416000000001]
----------------------------------------------
['Kiskon Technologies', 'Watry Design', 'Konica Minolta Business Solutions', 'Home Expo &  Design', 'Screened Magic', 'Summit Cabinet', 'The Ceramic Corporation', 'Woodenbridge', 'Bay Stone', 'Atlas Glass', 'Granite & Stone Expo', 'Zeus Tile', 'Emser Tile&Natural Stone', 'All Architectural Stone']


In [69]:
dist_design=design4["response"]["groups"][0]["items"][0]["venue"]["location"]["distance"]
lat_design=design4["response"]["groups"][0]["items"][0]["venue"]["location"]["lat"]
lng_design=design4["response"]["groups"][0]["items"][0]["venue"]["location"]["lng"]
name_design=design4["response"]["groups"][0]["items"][0]["venue"]["name"]

print(f"company name: {name_design}, company coordinates: {lat_design},{lng_design}, distance to the offices: {dist_design}")

company name: Kiskon Technologies, company coordinates: 37.37334604119023,-121.92193508148193, distance to the offices: 303


There is one design studio at 300m from the offices

In [70]:
# I make a query to the Foursquare API to look for schools near this company.

sch4=api_function('37.3721771, -121.9250398', 'Elementary School', 'elementary_sch_sj3')


In [71]:
print(dist(sch4))
print("----------------------------------------------")
print(lat(sch4))
print("----------------------------------------------")
print(lng(sch4))
print("----------------------------------------------")
print(name(sch4))

[1465, 2715, 2771, 3039, 3300, 3424, 3521, 3184, 3844, 3493, 4029, 4039, 4459, 4587]
----------------------------------------------
[37.36330687882652, 37.38955552112065, 37.388926007147695, 37.39309384415278, 37.358551025390625, 37.35738251403415, 37.343393216725616, 37.347336, 37.348379, 37.398261959325076, 37.38721347682061, 37.350991, 37.40608596801758, 37.333647875243265]
----------------------------------------------
[-121.9127954630632, -121.94658055857437, -121.90186142921448, -121.94713050142707, -121.95817565917969, -121.9589768845584, -121.94153776982016, -121.942885, -121.893555, -121.94699718220039, -121.88360782691444, -121.88796799999999, -121.898193359375, -121.94342527769899]
----------------------------------------------
['Bachrodt Elementary School', 'Montague Elementary School', 'Orchard Elementary School', 'Don Callejon School', 'Jano Hindi Bee', 'Scott Lane Elementary School', 'Washington Open Elementary', "St Clare's Catholic School", 'Suzume No Gakko', 'Don Call

In [72]:
dist_school=sch4["response"]["groups"][0]["items"][0]["venue"]["location"]["distance"]
lat_school=sch4["response"]["groups"][0]["items"][0]["venue"]["location"]["lat"]
lng_school=sch4["response"]["groups"][0]["items"][0]["venue"]["location"]["lng"]
name_school=sch4["response"]["groups"][0]["items"][0]["venue"]["name"]

print(f"company name: {name_school}, company coordinates: {lat_school},{lng_school}, distance to the offices: {dist_school}")

company name: Bachrodt Elementary School, company coordinates: 37.36330687882652,-121.9127954630632, distance to the offices: 1465


The nearest school is at 1.4km

In [73]:
# I make a query to the Foursquare API to look for nighlife spots near this company.

night4=api_function('37.3721771, -121.9250398', 'Nightlife Spot', 'nightlife_spot_sj3')


In [74]:
print(dist(night4))
print("----------------------------------------------")
print(lat(night4))
print("----------------------------------------------")
print(lng(night4))
print("----------------------------------------------")
print(name(night4))

[1932, 2804, 481, 1783, 710, 250, 270, 394, 419, 487, 599, 614, 557, 654]
----------------------------------------------
[37.36086599567488, 37.36132088609424, 37.37124135626228, 37.36381793355914, 37.365813193961756, 37.37152348661807, 37.371532546100354, 37.375498869419765, 37.375773, 37.369526, 37.37026696954015, 37.36690004129253, 37.37082598391839, 37.366398]
----------------------------------------------
[-121.90847504592831, -121.95364903471561, -121.91972331072563, -121.90784481894752, -121.9256078829507, -121.92232810486176, -121.9220969824811, -121.92348539829254, -121.923609, -121.929421, -121.93136993764425, -121.92706413888456, -121.93110972058392, -121.92641]
----------------------------------------------
['The Doghouse Sports Lounge', 'Golden State Brewery', 'Zone 8 Sports Bar & Grill', '4th St Bowl Lounge', 'Santa Cruz Wine Bar', 'Doubletree Pool', 'Fortyeight at DoubleTree', 'GSPS Wine Friday', 'Please Be Thirsty', 'San Jose Beer Union', "Chiaramonte's Saloon & Bar", '

In [75]:
dist_party=night4["response"]["groups"][0]["items"][5]["venue"]["location"]["distance"]
lat_party=night4["response"]["groups"][0]["items"][5]["venue"]["location"]["lat"]
lng_party=night4["response"]["groups"][0]["items"][5]["venue"]["location"]["lng"]
name_party=night4["response"]["groups"][0]["items"][5]["venue"]["name"]

print(f"company name: {name_party}, company coordinates: {lat_party},{lng_party}, distance to the offices: {dist_party}")

dist_party=night4["response"]["groups"][0]["items"][6]["venue"]["location"]["distance"]
lat_party=night4["response"]["groups"][0]["items"][6]["venue"]["location"]["lat"]
lng_party=night4["response"]["groups"][0]["items"][6]["venue"]["location"]["lng"]
name_party=night4["response"]["groups"][0]["items"][6]["venue"]["name"]

print(f"company name: {name_party}, company coordinates: {lat_party},{lng_party}, distance to the offices: {dist_party}")

company name: Doubletree Pool, company coordinates: 37.37152348661807,-121.92232810486176, distance to the offices: 250
company name: Fortyeight at DoubleTree, company coordinates: 37.371532546100354,-121.9220969824811, distance to the offices: 270


There are two places to go to party near the offices

In [80]:
# I make a query to the Foursquare API to look for vegetarian restaurants near this company.

vegan4=api_function('37.3721771, -121.9250398', 'Vegan Restaurant', 'veg_rest_sj3')


In [98]:
# There are 12 results for the vegan restaurants so I have to make another function for this query
def distv(lista):
    distances=[]
    for x in range(0,11):
        result=lista["response"]["groups"][0]["items"][x]["venue"]["location"]["distance"]
        distances.append(result)
    return distances

def latv(lista):
    latitude=[]
    for x in range(0,11):
        result=lista["response"]["groups"][0]["items"][x]["venue"]["location"]["lat"]
        latitude.append(result)
    return latitude

def lngv(lista):
    longitude=[]
    for x in range(0,11):
        result=lista["response"]["groups"][0]["items"][x]["venue"]["location"]["lng"]
        longitude.append(result)
    return longitude

def namev(lista):
    names=[]
    for x in range(0,11):
        result=lista["response"]["groups"][0]["items"][x]["venue"]["name"]
        names.append(result)
    return names

In [99]:
print(distv(vegan4))
print("----------------------------------------------")
print(latv(vegan4))
print("----------------------------------------------")
print(lngv(vegan4))
print("----------------------------------------------")
print(namev(vegan4))

[3146, 3457, 5401, 2110, 2602, 2614, 3539, 3843, 3992, 4024, 4157]
----------------------------------------------
[37.34460383642609, 37.343718446032305, 37.32614245482791, 37.385706, 37.35659, 37.3607916, 37.34049606323242, 37.33930772, 37.404856, 37.338702000000005, 37.396342]
----------------------------------------------
[-121.93286117841694, -121.9406861466985, -121.94432964084655, -121.908328, -121.94696, -121.8991962, -121.92169952392578, -121.93835143, -121.906444, -121.942205, -121.88920700000001]
----------------------------------------------
["Ike's Sandwiches", "deedee's", 'Loving Hut', 'Vegefarm', 'The Happy Hooligans', "Brother Baby's BBQ", 'Queen Of Sheba', 'Raw Vittles', 'Tasty Vegetarian Cuisine', 'Clean Eating with Katie', 'SJ Vegetarian']


In [100]:
dist_vegan=vegan4["response"]["groups"][0]["items"][4]["venue"]["location"]["distance"]
lat_vegan=vegan4["response"]["groups"][0]["items"][4]["venue"]["location"]["lat"]
lng_vegan=vegan4["response"]["groups"][0]["items"][4]["venue"]["location"]["lng"]
name_vegan=vegan4["response"]["groups"][0]["items"][4]["venue"]["name"]

print(f"company name: {name_vegan}, company coordinates: {lat_vegan},{lng_vegan}, distance to the offices: {dist_vegan}")

company name: The Happy Hooligans, company coordinates: 37.35659,-121.94696, distance to the offices: 2602


The nearest vegan restaurant is at 2.4km

In [82]:
# I make a query to the Foursquare API to look for basketball courts near this company.

basket4=api_function('37.3721771, -121.9250398', 'Basketball Court', 'basketball_court_sj3')


In [83]:
print(dist(basket4))
print("----------------------------------------------")
print(lat(basket4))
print("----------------------------------------------")
print(lng(basket4))
print("----------------------------------------------")
print(name(basket4))

[3854, 572, 1208, 1221, 1305, 2571, 2865, 3378, 3831, 3946, 4494, 4393, 4618, 4911]
----------------------------------------------
[37.33779578622328, 37.37696075439453, 37.37117385864258, 37.36441421508789, 37.364402, 37.3881680172003, 37.377344606195855, 37.39918075740896, 37.37372, 37.3753662109375, 37.40387815016547, 37.33270914406301, 37.330723171109355, 37.41575241088867]
----------------------------------------------
[-121.91989631992244, -121.92266082763672, -121.91143798828125, -121.91527557373047, -121.913997, -121.94602505794903, -121.9567734963286, -121.94248645305073, -121.968306, -121.96947479248047, -121.9565124356151, -121.9253028797591, -121.92286373936807, -121.91634368896484]
----------------------------------------------
['Central YMCA', 'eBay Inc. Basketball Court', 'Basketball Gym', 'Brocade Courts', 'Skyport!', 'Montague Park', '3 Ring', 'mansion grove basketball court', "Rucker's Basketball Court", 'Podium Golden State Warriors Basketball Court', 'Lick Mill Park

In [84]:
dist_basket=basket4["response"]["groups"][0]["items"][1]["venue"]["location"]["distance"]
lat_basket=basket4["response"]["groups"][0]["items"][1]["venue"]["location"]["lat"]
lng_basket=basket4["response"]["groups"][0]["items"][1]["venue"]["location"]["lng"]
name_basket=basket4["response"]["groups"][0]["items"][1]["venue"]["name"]

print(f"company name: {name_basket}, company coordinates: {lat_basket},{lng_basket}, distance to the offices: {dist_basket}")

company name: eBay Inc. Basketball Court, company coordinates: 37.37696075439453,-121.92266082763672, distance to the offices: 572


The basketball court is at 600m from the offices

In [85]:
# I make a query to the Foursquare API to look for Starbucks near this company.

starbucks4=api_function('37.3721771, -121.9250398', 'starbucks', 'starbucks_sj3')


In [86]:
print(dist(starbucks4))
print("----------------------------------------------")
print(lat(starbucks4))
print("----------------------------------------------")
print(lng(starbucks4))
print("----------------------------------------------")
print(name(starbucks4))

[701, 230, 492, 1278, 525, 1491, 3568, 3100, 2895, 3155, 4230, 2415, 4093, 3348]
----------------------------------------------
[37.37092547570862, 37.371585845947266, 37.369529724121094, 37.36107375737311, 37.369186, 37.385382, 37.40197092074824, 37.395157, 37.398163, 37.345295, 37.33418343, 37.35060173, 37.406467, 37.39293751646301]
----------------------------------------------
[-121.91727116354467, -121.92254638671875, -121.92949676513672, -121.92873647557042, -121.92045, -121.927919, -121.93991940591599, -121.90523999999999, -121.923706, -121.936371, -121.9241052, -121.9221276, -121.94176, -121.95243536434629]
----------------------------------------------
['Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks']


In [87]:
dist_starbucks=starbucks4["response"]["groups"][0]["items"][1]["venue"]["location"]["distance"]
lat_starbucks=starbucks4["response"]["groups"][0]["items"][1]["venue"]["location"]["lat"]
lng_starbucks=starbucks4["response"]["groups"][0]["items"][1]["venue"]["location"]["lng"]
name_starbucks=starbucks4["response"]["groups"][0]["items"][1]["venue"]["name"]

print(f"company name: {name_starbucks}, company coordinates: {lat_starbucks},{lng_starbucks}, distance to the offices: {dist_starbucks}")

dist_starbucks=starbucks4["response"]["groups"][0]["items"][2]["venue"]["location"]["distance"]
lat_starbucks=starbucks4["response"]["groups"][0]["items"][2]["venue"]["location"]["lat"]
lng_starbucks=starbucks4["response"]["groups"][0]["items"][2]["venue"]["location"]["lng"]
name_starbucks=starbucks4["response"]["groups"][0]["items"][2]["venue"]["name"]

print(f"company name: {name_starbucks}, company coordinates: {lat_starbucks},{lng_starbucks}, distance to the offices: {dist_starbucks}")

company name: Starbucks, company coordinates: 37.371585845947266,-121.92254638671875, distance to the offices: 230
company name: Starbucks, company coordinates: 37.369529724121094,-121.92949676513672, distance to the offices: 492


There are two Starbucks near

In [88]:
# I make a query to the Foursquare API to look for pet hairdressers near this company.

pet4=api_function('37.3721771, -121.9250398', 'Pet Service', 'pet_service_sj3')


In [89]:
print(dist(pet4))
print("----------------------------------------------")
print(lat(pet4))
print("----------------------------------------------")
print(lng(pet4))
print("----------------------------------------------")
print(name(pet4))

[3899, 2771, 3866, 5033, 2609, 2658, 2672, 2675, 2955, 3215, 3266, 3550, 3707, 3724]
----------------------------------------------
[37.339942, 37.3813788, 37.3402263, 37.35306842721221, 37.36269, 37.36273680000001, 37.3796526024096, 37.379786, 37.34972481944317, 37.38021, 37.40111, 37.403584, 37.34043631759206, 37.343559452656756]
----------------------------------------------
[-121.907791, -121.8959331, -121.9079148, -121.97660516837168, -121.95201, -121.9526368, -121.95375089364178, -121.953731, -121.90721511840819, -121.95995900000001, -121.91887, -121.932067, -121.91236495971678, -121.94683830386161]
----------------------------------------------
['PetSmart', 'Banfield Pet Hospital', 'Banfield Pet Hospital', 'Petco', 'Ohana Kennels', 'Bay Area K9 Association LLC.', 'A Pet Villa', 'A Pet Villa', 'DogWatch of the Bay Area & Northern California', 'Wag Hotels', 'Epic Dog Park', 'Dog Wash', 'Primal Canine', "Juan's Groom Room"]


In [90]:
dist_dog=pet4["response"]["groups"][0]["items"][4]["venue"]["location"]["distance"]
lat_dog=pet4["response"]["groups"][0]["items"][4]["venue"]["location"]["lat"]
lng_dog=pet4["response"]["groups"][0]["items"][4]["venue"]["location"]["lng"]
name_dog=pet4["response"]["groups"][0]["items"][4]["venue"]["name"]

print(f"company name: {name_dog}, company coordinates: {lat_dog},{lng_dog}, distance to the offices: {dist_dog}")

company name: Ohana Kennels, company coordinates: 37.36269,-121.95201, distance to the offices: 2609


The pet hairdresser is at 2.6km from the company

-----

# Fifth company analysis

In [ ]:
# I make a query to the Foursquare API to look for tech startups near this company.

tech5=api_function('37.3706126,-121.9219538', 'Tech Startup', 'tech_startups_sj4')


In [227]:
print(dist(tech5))
print("----------------------------------------------")
print(lat(tech5))
print("----------------------------------------------")
print(lng(tech5))
print("----------------------------------------------")
print(name(tech5))

[593, 157, 118, 133, 187, 315, 328, 375, 450, 473, 478, 487, 560, 576]
----------------------------------------------
[37.375874679466456, 37.369302000000005, 37.36957044583222, 37.3695173976203, 37.36947, 37.370972648019674, 37.36805224766053, 37.36753085454393, 37.368907, 37.36766592556348, 37.367859, 37.368927, 37.36666122235786, 37.36609659285964]
----------------------------------------------
[-121.92306376202592, -121.92127099999999, -121.921677387159, -121.92133577652343, -121.92351, -121.92549178587224, -121.92011894731519, -121.92021896118176, -121.917341, -121.9180963651028, -121.917805, -121.91687, -121.91802978515625, -121.91876875122978]
----------------------------------------------
['PayPal', 'PlayPhone, Inc.', 'River Meadow Software', 'Robin Systems', 'FixStream', 'SourceN HQ', 'Gaia Online', 'Ensighten', 'GEO Seminconductor', 'Nutanix HQ', 'Usens', 'XtremIO', 'Ziffity Solutions LLC', 'Calypto Design Systems']


In [99]:
dist_tech=tech5["response"]["groups"][0]["items"][1]["venue"]["location"]["distance"]
lat_tech=tech5["response"]["groups"][0]["items"][1]["venue"]["location"]["lat"]
lng_tech=tech5["response"]["groups"][0]["items"][1]["venue"]["location"]["lng"]
name_tech=tech5["response"]["groups"][0]["items"][1]["venue"]["name"]

print(f"company name: {name_tech}, company coordinates: {lat_tech},{lng_tech}, distance to the offices: {dist_tech}")

dist_tech=tech5["response"]["groups"][0]["items"][2]["venue"]["location"]["distance"]
lat_tech=tech5["response"]["groups"][0]["items"][2]["venue"]["location"]["lat"]
lng_tech=tech5["response"]["groups"][0]["items"][2]["venue"]["location"]["lng"]
name_tech=tech5["response"]["groups"][0]["items"][2]["venue"]["name"]

print(f"company name: {name_tech}, company coordinates: {lat_tech},{lng_tech}, distance to the offices: {dist_tech}")


dist_tech=tech5["response"]["groups"][0]["items"][3]["venue"]["location"]["distance"]
lat_tech=tech5["response"]["groups"][0]["items"][3]["venue"]["location"]["lat"]
lng_tech=tech5["response"]["groups"][0]["items"][3]["venue"]["location"]["lng"]
name_tech=tech5["response"]["groups"][0]["items"][3]["venue"]["name"]

print(f"company name: {name_tech}, company coordinates: {lat_tech},{lng_tech}, distance to the offices: {dist_tech}")

dist_tech=tech5["response"]["groups"][0]["items"][4]["venue"]["location"]["distance"]
lat_tech=tech5["response"]["groups"][0]["items"][4]["venue"]["location"]["lat"]
lng_tech=tech5["response"]["groups"][0]["items"][4]["venue"]["location"]["lng"]
name_tech=tech5["response"]["groups"][0]["items"][4]["venue"]["name"]

print(f"company name: {name_tech}, company coordinates: {lat_tech},{lng_tech}, distance to the offices: {dist_tech}")

company name: PlayPhone, Inc., company coordinates: 37.369302000000005,-121.92127099999999, distance to the offices: 157
company name: River Meadow Software, company coordinates: 37.36957044583222,-121.921677387159, distance to the offices: 118
company name: Robin Systems, company coordinates: 37.3695173976203,-121.92133577652343, distance to the offices: 133
company name: FixStream, company coordinates: 37.36947,-121.92351, distance to the offices: 187


There are 4 tech startups very close to the offices

In [ ]:
# I make a query to the Foursquare API to look for design studios near this company.

design5=api_function('37.3706126,-121.9219538', 'Design Studio', 'design_studio_sj4')


In [226]:
print(dist(design5))
print("----------------------------------------------")
print(lat(design5))
print("----------------------------------------------")
print(lng(design5))
print("----------------------------------------------")
print(name(design5))

[304, 643, 434, 1134, 1203, 1216, 1231, 1231, 1339, 1418, 1458, 1485, 1255, 1725]
----------------------------------------------
[37.37334604119023, 37.3672899, 37.3691219, 37.37102127075195, 37.3707160949707, 37.378426714690484, 37.374528, 37.373359, 37.374013, 37.379857, 37.373688, 37.373843, 37.3657117, 37.383968353271484]
----------------------------------------------
[-121.92193508148193, -121.916, -121.917411, -121.90914154052734, -121.9083480834961, -121.91234536963067, -121.908934, -121.90847, -121.907427, -121.910925, -121.905933, -121.905661, -121.9091752, -121.91206359863281]
----------------------------------------------
['Kiskon Technologies', 'Watry Design', 'Konica Minolta Business Solutions', 'Screened Magic', 'Woodenbridge', 'Home Expo &  Design', 'Summit Cabinet', 'The Ceramic Corporation', 'Atlas Glass', 'Bay Stone', 'Granite & Stone Expo', 'Zeus Tile', 'Signarama San Jose', 'Emser Tile&Natural Stone']


In [103]:
dist_design=design5["response"]["groups"][0]["items"][0]["venue"]["location"]["distance"]
lat_design=design5["response"]["groups"][0]["items"][0]["venue"]["location"]["lat"]
lng_design=design5["response"]["groups"][0]["items"][0]["venue"]["location"]["lng"]
name_design=design5["response"]["groups"][0]["items"][0]["venue"]["name"]

print(f"company name: {name_design}, company coordinates: {lat_design},{lng_design}, distance to the offices: {dist_design}")

dist_design=design5["response"]["groups"][0]["items"][2]["venue"]["location"]["distance"]
lat_design=design5["response"]["groups"][0]["items"][2]["venue"]["location"]["lat"]
lng_design=design5["response"]["groups"][0]["items"][2]["venue"]["location"]["lng"]
name_design=design5["response"]["groups"][0]["items"][2]["venue"]["name"]

print(f"company name: {name_design}, company coordinates: {lat_design},{lng_design}, distance to the offices: {dist_design}")

company name: Kiskon Technologies, company coordinates: 37.37334604119023,-121.92193508148193, distance to the offices: 304
company name: Konica Minolta Business Solutions, company coordinates: 37.3691219,-121.917411, distance to the offices: 434


The nearest studios are at 300m and 400m

In [ ]:
# I make a query to the Foursquare API to look for schools near this company.

sch5=api_function('37.3706126,-121.9219538', 'Elementary School', 'elementary_sch_sj4')


In [225]:
print(dist(sch5))
print("----------------------------------------------")
print(lat(sch5))
print("----------------------------------------------")
print(lng(sch5))
print("----------------------------------------------")
print(name(sch5))

[1148, 2704, 3031, 3349, 3474, 3490, 3526, 3184, 3591, 3716, 3862, 3792, 4413, 4457]
----------------------------------------------
[37.36330687882652, 37.388926007147695, 37.38955552112065, 37.39309384415278, 37.358551025390625, 37.343393216725616, 37.348379, 37.347336, 37.35738251403415, 37.350991, 37.38721347682061, 37.398261959325076, 37.35531655464134, 37.379878999999995]
----------------------------------------------
[-121.9127954630632, -121.90186142921448, -121.94658055857437, -121.94713050142707, -121.95817565917969, -121.94153776982016, -121.893555, -121.942885, -121.9589768845584, -121.88796799999999, -121.88360782691444, -121.94699718220039, -121.87593729046975, -121.872928]
----------------------------------------------
['Bachrodt Elementary School', 'Orchard Elementary School', 'Montague Elementary School', 'Don Callejon School', 'Jano Hindi Bee', 'Washington Open Elementary', 'Suzume No Gakko', "St Clare's Catholic School", 'Scott Lane Elementary School', 'Grant Elementa

In [106]:
dist_school=sch5["response"]["groups"][0]["items"][0]["venue"]["location"]["distance"]
lat_school=sch5["response"]["groups"][0]["items"][0]["venue"]["location"]["lat"]
lng_school=sch5["response"]["groups"][0]["items"][0]["venue"]["location"]["lng"]
name_school=sch5["response"]["groups"][0]["items"][0]["venue"]["name"]

print(f"company name: {name_school}, company coordinates: {lat_school},{lng_school}, distance to the offices: {dist_school}")

company name: Bachrodt Elementary School, company coordinates: 37.36330687882652,-121.9127954630632, distance to the offices: 1148


There is a school at 1.1km from the office

In [ ]:
# I make a query to the Foursquare API to look for nighlife spots near this company.

night5=api_function('37.3706126,-121.9219538', 'Nightlife Spot', 'nightlife_spot_sj4')


In [224]:
print(dist(night5))
print("----------------------------------------------")
print(lat(night5))
print("----------------------------------------------")
print(lng(night5))
print("----------------------------------------------")
print(name(night5))

[1612, 209, 1459, 624, 106, 103, 560, 592, 2174, 612, 612, 644, 657, 671]
----------------------------------------------
[37.36086599567488, 37.37124135626228, 37.36381793355914, 37.365813193961756, 37.37152348661807, 37.371532546100354, 37.375498869419765, 37.375773, 37.35856138262397, 37.366398, 37.36690004129253, 37.36545102537579, 37.3700879, 37.369526]
----------------------------------------------
[-121.90847504592831, -121.91972331072563, -121.90784481894752, -121.9256078829507, -121.92232810486176, -121.9220969824811, -121.92348539829254, -121.923609, -121.90260409915966, -121.92641, -121.92706413888456, -121.9186465104381, -121.9145503, -121.929421]
----------------------------------------------
['The Doghouse Sports Lounge', 'Zone 8 Sports Bar & Grill', '4th St Bowl Lounge', 'Santa Cruz Wine Bar', 'Doubletree Pool', 'Fortyeight at DoubleTree', 'GSPS Wine Friday', 'Please Be Thirsty', 'Hyde Park Lounge', 'T Baggers', 'shark cage', 'the bistro', "The Arms' Ride", 'San Jose Beer

In [109]:
dist_party=night5["response"]["groups"][0]["items"][1]["venue"]["location"]["distance"]
lat_party=night5["response"]["groups"][0]["items"][1]["venue"]["location"]["lat"]
lng_party=night5["response"]["groups"][0]["items"][1]["venue"]["location"]["lng"]
name_party=night5["response"]["groups"][0]["items"][1]["venue"]["name"]

print(f"company name: {name_party}, company coordinates: {lat_party},{lng_party}, distance to the offices: {dist_party}")

dist_party=night5["response"]["groups"][0]["items"][4]["venue"]["location"]["distance"]
lat_party=night5["response"]["groups"][0]["items"][4]["venue"]["location"]["lat"]
lng_party=night5["response"]["groups"][0]["items"][4]["venue"]["location"]["lng"]
name_party=night5["response"]["groups"][0]["items"][4]["venue"]["name"]

print(f"company name: {name_party}, company coordinates: {lat_party},{lng_party}, distance to the offices: {dist_party}")

dist_party=night5["response"]["groups"][0]["items"][5]["venue"]["location"]["distance"]
lat_party=night5["response"]["groups"][0]["items"][5]["venue"]["location"]["lat"]
lng_party=night5["response"]["groups"][0]["items"][5]["venue"]["location"]["lng"]
name_party=night5["response"]["groups"][0]["items"][5]["venue"]["name"]

print(f"company name: {name_party}, company coordinates: {lat_party},{lng_party}, distance to the offices: {dist_party}")

company name: Zone 8 Sports Bar & Grill, company coordinates: 37.37124135626228,-121.91972331072563, distance to the offices: 209
company name: Doubletree Pool, company coordinates: 37.37152348661807,-121.92232810486176, distance to the offices: 106
company name: Fortyeight at DoubleTree, company coordinates: 37.371532546100354,-121.9220969824811, distance to the offices: 103


There are 3 nighlife spots near the offices

In [ ]:
# I make a query to the Foursquare API to look for vegetarian restaurants near this company.

vegan5=api_function('37.3706126,-121.9219538', 'Vegan Restaurant', 'veg_rest_sj4')



In [223]:
print(dist(vegan5))
print("----------------------------------------------")
print(lat(vegan5))
print("----------------------------------------------")
print(lng(vegan5))
print("----------------------------------------------")
print(name(vegan5))

[3051, 4821, 4733, 4987, 3422, 4892, 5484, 5331, 2067, 2291, 2707, 3352, 3774, 3978]
----------------------------------------------
[37.34460383642609, 37.33547578302974, 37.336852209398536, 37.341433051588915, 37.343718446032305, 37.33993504196706, 37.330296164668816, 37.32614245482791, 37.385706, 37.3607916, 37.35659, 37.34049606323242, 37.33930772, 37.338702000000005]
----------------------------------------------
[-121.93286117841694, -121.89009385118939, -121.88943603809092, -121.87917600579642, -121.9406861466985, -121.88235410455921, -121.88632717974279, -121.94432964084655, -121.908328, -121.8991962, -121.94696, -121.92169952392578, -121.93835143, -121.942205]
----------------------------------------------
["Ike's Sandwiches", 'Good Karma', "Ike's Sandwiches", 'Vegetarian House', "deedee's", 'Tofoo Com Chay', 'Vitamina Juices & Blends', 'Loving Hut', 'Vegefarm', "Brother Baby's BBQ", 'The Happy Hooligans', 'Queen Of Sheba', 'Raw Vittles', 'Clean Eating with Katie']


In [114]:
dist_vegan=vegan5["response"]["groups"][0]["items"][8]["venue"]["location"]["distance"]
lat_vegan=vegan5["response"]["groups"][0]["items"][8]["venue"]["location"]["lat"]
lng_vegan=vegan5["response"]["groups"][0]["items"][8]["venue"]["location"]["lng"]
name_vegan=vegan5["response"]["groups"][0]["items"][8]["venue"]["name"]

print(f"company name: {name_vegan}, company coordinates: {lat_vegan},{lng_vegan}, distance to the offices: {dist_vegan}")

company name: Vegefarm, company coordinates: 37.385706,-121.908328, distance to the offices: 2067


The nearest vegan restaurant is at 2km

In [ ]:
# I make a query to the Foursquare API to look for basketball courts near this company.

basket5=api_function('37.3706126,-121.9219538', 'Basketball Court', 'basketball_court_sj4')


In [222]:
print(dist(basket5))
print("----------------------------------------------")
print(lat(basket5))
print("----------------------------------------------")
print(lng(basket5))
print("----------------------------------------------")
print(name(basket5))

[3657, 709, 908, 932, 986, 2890, 3170, 3662, 4115, 4229, 4237, 4801, 4441, 4713]
----------------------------------------------
[37.33779578622328, 37.37696075439453, 37.36441421508789, 37.37117385864258, 37.364402, 37.3881680172003, 37.377344606195855, 37.39918075740896, 37.37372, 37.33270914406301, 37.3753662109375, 37.40387815016547, 37.330723171109355, 37.3980712890625]
----------------------------------------------
[-121.91989631992244, -121.92266082763672, -121.91527557373047, -121.91143798828125, -121.913997, -121.94602505794903, -121.9567734963286, -121.94248645305073, -121.968306, -121.9253028797591, -121.96947479248047, -121.9565124356151, -121.92286373936807, -121.88139343261719]
----------------------------------------------
['Central YMCA', 'eBay Inc. Basketball Court', 'Brocade Courts', 'Basketball Gym', 'Skyport!', 'Montague Park', '3 Ring', 'mansion grove basketball court', "Rucker's Basketball Court", 'Hoover Gym', 'Podium Golden State Warriors Basketball Court', 'Lick

In [119]:
dist_basket=basket5["response"]["groups"][0]["items"][1]["venue"]["location"]["distance"]
lat_basket=basket5["response"]["groups"][0]["items"][1]["venue"]["location"]["lat"]
lng_basket=basket5["response"]["groups"][0]["items"][1]["venue"]["location"]["lng"]
name_basket=basket5["response"]["groups"][0]["items"][1]["venue"]["name"]

print(f"company name: {name_basket}, company coordinates: {lat_basket},{lng_basket}, distance to the offices: {dist_basket}")

dist_basket=basket5["response"]["groups"][0]["items"][2]["venue"]["location"]["distance"]
lat_basket=basket5["response"]["groups"][0]["items"][2]["venue"]["location"]["lat"]
lng_basket=basket5["response"]["groups"][0]["items"][2]["venue"]["location"]["lng"]
name_basket=basket5["response"]["groups"][0]["items"][2]["venue"]["name"]

print(f"company name: {name_basket}, company coordinates: {lat_basket},{lng_basket}, distance to the offices: {dist_basket}")

company name: eBay Inc. Basketball Court, company coordinates: 37.37696075439453,-121.92266082763672, distance to the offices: 709
company name: Brocade Courts, company coordinates: 37.36441421508789,-121.91527557373047, distance to the offices: 908


There are two basketball courts near the office

In [ ]:
# I make a query to the Foursquare API to look for Starbucks near this company.

starbucks5=api_function('37.3706126,-121.9219538', 'starbucks', 'starbucks_sj4')


In [221]:
print(dist(starbucks5))
print("----------------------------------------------")
print(lat(starbucks5))
print("----------------------------------------------")
print(lng(starbucks5))
print("----------------------------------------------")
print(name(starbucks5))

[415, 120, 678, 1219, 207, 4059, 2227, 3093, 3106, 3374, 2812, 3835, 1726, 3070]
----------------------------------------------
[37.37092547570862, 37.371585845947266, 37.369529724121094, 37.36107375737311, 37.369186, 37.33418343, 37.35060173, 37.345295, 37.395157, 37.341431, 37.38324096, 37.40197092074824, 37.385382, 37.398163]
----------------------------------------------
[-121.91727116354467, -121.92254638671875, -121.92949676513672, -121.92873647557042, -121.92045, -121.9241052, -121.9221276, -121.936371, -121.90523999999999, -121.911642, -121.894416, -121.93991940591599, -121.927919, -121.923706]
----------------------------------------------
['Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks']


In [122]:
dist_starbucks=starbucks5["response"]["groups"][0]["items"][1]["venue"]["location"]["distance"]
lat_starbucks=starbucks5["response"]["groups"][0]["items"][1]["venue"]["location"]["lat"]
lng_starbucks=starbucks5["response"]["groups"][0]["items"][1]["venue"]["location"]["lng"]
name_starbucks=starbucks5["response"]["groups"][0]["items"][1]["venue"]["name"]

print(f"company name: {name_starbucks}, company coordinates: {lat_starbucks},{lng_starbucks}, distance to the offices: {dist_starbucks}")

dist_starbucks=starbucks5["response"]["groups"][0]["items"][4]["venue"]["location"]["distance"]
lat_starbucks=starbucks5["response"]["groups"][0]["items"][4]["venue"]["location"]["lat"]
lng_starbucks=starbucks5["response"]["groups"][0]["items"][4]["venue"]["location"]["lng"]
name_starbucks=starbucks5["response"]["groups"][0]["items"][4]["venue"]["name"]

print(f"company name: {name_starbucks}, company coordinates: {lat_starbucks},{lng_starbucks}, distance to the offices: {dist_starbucks}")

company name: Starbucks, company coordinates: 37.371585845947266,-121.92254638671875, distance to the offices: 120
company name: Starbucks, company coordinates: 37.369186,-121.92045, distance to the offices: 207


There are two Starbucks at 120 and 200 km from the office

In [ ]:
# I make a query to the Foursquare API to look for pet hairdressers near this company.

pet5=api_function('37.3706126,-121.9219538', 'Pet Service', 'pet_service_sj4')


In [220]:
print(dist(pet5))
print("----------------------------------------------")
print(lat(pet5))
print("----------------------------------------------")
print(lng(pet5))
print("----------------------------------------------")
print(name(pet5))

[3669, 2777, 4780, 3636, 5028, 2613, 2664, 2827, 2831, 2729, 3363, 3463, 3478, 3592]
----------------------------------------------
[37.339942, 37.3813788, 37.32848854373601, 37.3402263, 37.35306842721221, 37.36269, 37.36273680000001, 37.3796526024096, 37.379786, 37.34972481944317, 37.38021, 37.40111, 37.34043631759206, 37.343559452656756]
----------------------------------------------
[-121.907791, -121.8959331, -121.91154708173961, -121.9079148, -121.97660516837168, -121.95201, -121.9526368, -121.95375089364178, -121.953731, -121.90721511840819, -121.95995900000001, -121.91887, -121.91236495971678, -121.94683830386161]
----------------------------------------------
['PetSmart', 'Banfield Pet Hospital', 'Showtime Pet Grooming', 'Banfield Pet Hospital', 'Petco', 'Ohana Kennels', 'Bay Area K9 Association LLC.', 'A Pet Villa', 'A Pet Villa', 'DogWatch of the Bay Area & Northern California', 'Wag Hotels', 'Epic Dog Park', 'Primal Canine', "Juan's Groom Room"]


In [125]:
dist_dog=pet5["response"]["groups"][0]["items"][1]["venue"]["location"]["distance"]
lat_dog=pet5["response"]["groups"][0]["items"][1]["venue"]["location"]["lat"]
lng_dog=pet5["response"]["groups"][0]["items"][1]["venue"]["location"]["lng"]
name_dog=pet5["response"]["groups"][0]["items"][1]["venue"]["name"]

print(f"company name: {name_dog}, company coordinates: {lat_dog},{lng_dog}, distance to the offices: {dist_dog}")

company name: Banfield Pet Hospital, company coordinates: 37.3813788,-121.8959331, distance to the offices: 2595


The nearest pet hairdresser is at 2.6km from the company